In [34]:
import calendar
import warnings
import pdfplumber
import re
from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

# Generación del texto a partir del PDF

In [35]:
def file_path_fcn(file_path=None):
    if file_path is None:
        file_path = 'REPORTES/KENA.pdf'
    return file_path

file_path ='REPORTES/KENA.pdf'

file_path = file_path_fcn()
file_path

'REPORTES/KENA.pdf'

In [36]:
def pdf_a_texto(file_path):
    '''Conversion del pdf a texto'''
    with pdfplumber.open(file_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

texto = pdf_a_texto(file_path)

## Convertir la fecha a español

In [50]:
from datetime import datetime

def format_spanish_date(date_str):
    # Define Spanish month and day names
    days = ["Lunes", "Martes", "Miércoles", "Jueves",
            "Viernes", "Sábado", "Domingo"]
    months = ["enero", "febrero", "marzo", "abril", "mayo", "junio", "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]

    # Parse the input date
    date_obj = datetime.strptime(date_str, "%d/%m/%Y")

    # Get day of week (0=Monday, 6=Sunday)
    day_of_week = date_obj.weekday()

    # Format the date in Spanish
    return f"{days[day_of_week]}, {date_obj.day} de {months[date_obj.month - 1]} de {date_obj.year}"

# Example usage:
date_str = "13/10/1964"
formatted_date = format_spanish_date(date_str)
print(formatted_date)  # Output: Martes, 13 de octubre de 1964

Martes, 13 de octubre de 1964


In [51]:
# Converting the general dates to datetime format
def convertir_a_fecha(fecha_str):
    # All the dates from the PDF are in the format "DD/MM/YYYY"
    fecha = datetime.strptime(fecha_str, '%d/%m/%Y').date()
    print(fecha)
    print(type(fecha))
    return fecha

# Example usage
esempio = "13/10/1964"
convertir_a_fecha(esempio)


1964-10-13
<class 'datetime.date'>


datetime.date(1964, 10, 13)

In [37]:
class DatosGenerales:
    def __init__(self, texto):
        self.texto = texto

    def nombrefcn(self):
    # Extract the substring between the found positions, adjust indices for correct slicing
        InicioStr = re.search(r"20\d{2}\s{1}", self.texto)
        FinalStr = "DD MM YYYY"
        # Find the starting and ending positions of the target substrings
        Inicio = InicioStr.span()[0]
        Final = texto.find(FinalStr)
        # Extract the substring between the found positions, adjust indices for correct slicing
        if Inicio != -1 and Final != -1:
            # + 5 since InicioStr length = length(20) + 2 + 1 = 5
            Nombre = self.texto[Inicio + 5: Final].strip()
        else:
            Nombre = ""
        print(f"NOMBRE: {Nombre}")
        self.Nombre = Nombre
        return Nombre

    def nssfcn(self):
        InicioStr = "NSS: "
        FinalStr = "CURP:"

        Inicio = self.texto.find(InicioStr)
        Final = self.texto.find(FinalStr)

        if Inicio != -1 and Final != -1:
            nss = self.texto[Inicio + 5: Final - 28].strip()
        else:
            nss = ""
        print(f"NSS: {nss}")
        self.nss = nss
        return nss

    def curpfcn(self):
        InicioStr = "CURP:"

        Inicio = self.texto.find(InicioStr)

        if Inicio != -1:
            curp = self.texto[Inicio +len(InicioStr): Inicio + 25].strip()
        else:
            curp = ''
        print(f"CURP: {curp}")
        self.curp = curp
        return curp


    def fecha_nacimiento_fcn(self):
        YEAR_BIRTH = "19" + self.curp[4:6]
        MONTH_BIRTH = self.curp[6:8]
        DAY_BIRTH = self.curp[8:10]
        FECHA_NACIMIENTO = DAY_BIRTH + '/' + MONTH_BIRTH + '/' + YEAR_BIRTH
        # Convert string to datetime object
        print(FECHA_NACIMIENTO)
        print(type(FECHA_NACIMIENTO))
        FECHA_NACIMIENTO = datetime.strptime(FECHA_NACIMIENTO, '%d/%m/%Y').date()
        print(f"FECHA DE NACIMIENTO: {FECHA_NACIMIENTO.strftime('%A %d %b %Y')}")
        self.fecha_nacimiento = FECHA_NACIMIENTO
        return FECHA_NACIMIENTO.strftime('%A %d %b %Y'), FECHA_NACIMIENTO

    def ano_inscripcionIMSS_fcn(self):
        TerminacionYear = self.nss[2:4]
        if int(TerminacionYear) < 50:
            AnoInicio = int('20'+ TerminacionYear)
        else:
            AnoInicio = int('19'+ TerminacionYear)
        print(f"AÑO DE INSCRIPCIÓN IMSS: {AnoInicio}")
        self.AnoInicio = AnoInicio
        return AnoInicio

    def regimen_pertenecefcn(self):
        if self.AnoInicio > 1997:
            Regimen = 'Régimen 97'
        elif self.AnoInicio == 1997:
            Regimen = "Determinar el mes que inició, si es a partir de julio, entra en régimen 97"
        else:
            Regimen = 'Régimen 73'

        print(f"REGIMEN AL QUE PERTENECE: {Regimen}")
        self.Regimen = Regimen
        return Regimen

    def semanas_cotizadas_descontadas_fcn(self):
        Iniciostr = re.search(r"\d{1,4}\s{1}\d{1,4}\s{1}\d{1,4}", texto)
        myrange = Iniciostr.span()
        risultati = self.texto[myrange[0]:myrange[1]]
        semanas_cotizadas = int(risultati.split()[0])
        semanas_descontadas = int(risultati.split()[1])
        semanas_reintegradas = int(risultati.split()[-1])
        semanas_totales = semanas_cotizadas - semanas_descontadas + semanas_reintegradas

        print(f"SEMANAS COTIZADAS: {semanas_cotizadas}")
        print(f"SEMANAS DESCONTADAS: {semanas_descontadas}")
        print(f"SEMANAS REINTEGRADAS: {semanas_reintegradas}")
        print(f"SEMANAS TOTALES: {semanas_totales}")

        self.semanas_cotizadas = semanas_cotizadas
        self.semanas_descontadas = semanas_descontadas
        self.semanas_reintegradas = semanas_reintegradas
        self.semanas_totales = semanas_totales

        return semanas_cotizadas, semanas_descontadas, semanas_reintegradas


    def fecha_emision_reporte_fcn(self):
        texto_array = self.texto.split('\n')

        # Find the position of the string that ends with 'reporte' and select the next element that contains the date
        position_date_report = [i + 1 for i, s in enumerate(texto_array) if s.endswith('reporte')]
        out = texto_array[position_date_report[0]]
        out = out.replace(' ', '')
        out = datetime.strptime(out, '%d/%m/%Y').date()
        print(f"FECHA DE EMISIÓN DEL REPORTE: {out}")
        self.fecha_emision_reporte = out
        return out

    def exportar_datos(self, name):
        out = pd.DataFrame({
            'NOMBRE': [self.Nombre],
            'NSS': [self.nss],
            'CURP': [self.curp],
            'FECHA DE NACIMIENTO': [self.fecha_nacimiento],
            'AÑO DE INSCRIPCIÓN IMSS': [self.AnoInicio],
            'REGIMEN AL QUE PERTENECE': [self.Regimen],
            'SEMANAS COTIZADAS': [self.semanas_cotizadas],
            'SEMANAS DESCONTADAS': [self.semanas_descontadas],
            'SEMANAS REINTEGRADAS': [self.semanas_reintegradas],
            'SEMANAS TOTALES': [self.semanas_totales],
            'FECHA DE EMISIÓN DEL REPORTE': [self.fecha_emision_reporte]
        }
        )

        out.to_csv(name)
        return out

    def tabla_datos(self):
        out = pd.DataFrame({
            'NOMBRE': [self.Nombre],
            'NSS': [self.nss],
            'CURP': [self.curp],
            'FECHA DE NACIMIENTO': [self.fecha_nacimiento],
            'AÑO DE INSCRIPCIÓN IMSS': [self.AnoInicio],
            'REGIMEN AL QUE PERTENECE': [self.Regimen],
            'SEMANAS COTIZADAS': [self.semanas_cotizadas],
            'SEMANAS DESCONTADAS': [self.semanas_descontadas],
            'SEMANAS REINTEGRADAS': [self.semanas_reintegradas],
            'SEMANAS TOTALES': [self.semanas_totales],
            'FECHA DE EMISIÓN DEL REPORTE': [self.fecha_emision_reporte]
        }
        )
        out = out.T
        out.columns = ['DATOS PERSONALES']
        return out

In [53]:
class DatosGenerales:
    def __init__(self, texto):
        self.texto = texto

    def nombrefcn(self):
    # Extract the substring between the found positions, adjust indices for correct slicing
        InicioStr = re.search(r"20\d{2}\s{1}", self.texto)
        FinalStr = "DD MM YYYY"
        # Find the starting and ending positions of the target substrings
        Inicio = InicioStr.span()[0]
        Final = texto.find(FinalStr)
        # Extract the substring between the found positions, adjust indices for correct slicing
        if Inicio != -1 and Final != -1:
            # + 5 since InicioStr length = length(20) + 2 + 1 = 5
            Nombre = self.texto[Inicio + 5: Final].strip()
        else:
            Nombre = ""
        print(f"NOMBRE: {Nombre}")
        self.Nombre = Nombre
        return Nombre

    def nssfcn(self):
        InicioStr = "NSS: "
        FinalStr = "CURP:"

        Inicio = self.texto.find(InicioStr)
        Final = self.texto.find(FinalStr)

        if Inicio != -1 and Final != -1:
            nss = self.texto[Inicio + 5: Final - 28].strip()
        else:
            nss = ""
        print(f"NSS: {nss}")
        self.nss = nss
        return nss

    def curpfcn(self):
        InicioStr = "CURP:"

        Inicio = self.texto.find(InicioStr)

        if Inicio != -1:
            curp = self.texto[Inicio +len(InicioStr): Inicio + 25].strip()
        else:
            curp = ''
        print(f"CURP: {curp}")
        self.curp = curp
        return curp


    def fecha_nacimiento_fcn(self):
        YEAR_BIRTH = "19" + self.curp[4:6]
        MONTH_BIRTH = self.curp[6:8]
        DAY_BIRTH = self.curp[8:10]
        FECHA_NACIMIENTO = DAY_BIRTH + '/' + MONTH_BIRTH + '/' + YEAR_BIRTH
        # Convert string to datetime object
        # FECHA_NACIMIENTO = datetime.strptime(FECHA_NACIMIENTO, '%d/%m/%Y').date()
        print(FECHA_NACIMIENTO)
        print(type(FECHA_NACIMIENTO))
        FECHA_NACIMIENTO_num = convertir_a_fecha(FECHA_NACIMIENTO)
        print(FECHA_NACIMIENTO_num)
        print(type(FECHA_NACIMIENTO_num))
        # FECHA_NACIMIENTO =
        print(f"FECHA DE NACIMIENTO: {FECHA_NACIMIENTO.strftime('%A %d %b %Y')}")
        self.fecha_nacimiento = FECHA_NACIMIENTO
        return FECHA_NACIMIENTO.strftime('%A %d %b %Y'), FECHA_NACIMIENTO

    def ano_inscripcionIMSS_fcn(self):
        TerminacionYear = self.nss[2:4]
        if int(TerminacionYear) < 50:
            AnoInicio = int('20'+ TerminacionYear)
        else:
            AnoInicio = int('19'+ TerminacionYear)
        print(f"AÑO DE INSCRIPCIÓN IMSS: {AnoInicio}")
        self.AnoInicio = AnoInicio
        return AnoInicio

    def regimen_pertenecefcn(self):
        if self.AnoInicio > 1997:
            Regimen = 'Régimen 97'
        elif self.AnoInicio == 1997:
            Regimen = "Determinar el mes que inició, si es a partir de julio, entra en régimen 97"
        else:
            Regimen = 'Régimen 73'

        print(f"REGIMEN AL QUE PERTENECE: {Regimen}")
        self.Regimen = Regimen
        return Regimen

    def semanas_cotizadas_descontadas_fcn(self):
        Iniciostr = re.search(r"\d{1,4}\s{1}\d{1,4}\s{1}\d{1,4}", texto)
        myrange = Iniciostr.span()
        risultati = self.texto[myrange[0]:myrange[1]]
        semanas_cotizadas = int(risultati.split()[0])
        semanas_descontadas = int(risultati.split()[1])
        semanas_reintegradas = int(risultati.split()[-1])
        semanas_totales = semanas_cotizadas - semanas_descontadas + semanas_reintegradas

        print(f"SEMANAS COTIZADAS: {semanas_cotizadas}")
        print(f"SEMANAS DESCONTADAS: {semanas_descontadas}")
        print(f"SEMANAS REINTEGRADAS: {semanas_reintegradas}")
        print(f"SEMANAS TOTALES: {semanas_totales}")

        self.semanas_cotizadas = semanas_cotizadas
        self.semanas_descontadas = semanas_descontadas
        self.semanas_reintegradas = semanas_reintegradas
        self.semanas_totales = semanas_totales

        return semanas_cotizadas, semanas_descontadas, semanas_reintegradas


    def fecha_emision_reporte_fcn(self):
        texto_array = self.texto.split('\n')

        # Find the position of the string that ends with 'reporte' and select the next element that contains the date
        position_date_report = [i + 1 for i, s in enumerate(texto_array) if s.endswith('reporte')]
        out = texto_array[position_date_report[0]]
        out = out.replace(' ', '')
        out = datetime.strptime(out, '%d/%m/%Y').date()
        print(f"FECHA DE EMISIÓN DEL REPORTE: {out}")
        self.fecha_emision_reporte = out
        return out

    def exportar_datos(self, name):
        out = pd.DataFrame({
            'NOMBRE': [self.Nombre],
            'NSS': [self.nss],
            'CURP': [self.curp],
            'FECHA DE NACIMIENTO': [self.fecha_nacimiento],
            'AÑO DE INSCRIPCIÓN IMSS': [self.AnoInicio],
            'REGIMEN AL QUE PERTENECE': [self.Regimen],
            'SEMANAS COTIZADAS': [self.semanas_cotizadas],
            'SEMANAS DESCONTADAS': [self.semanas_descontadas],
            'SEMANAS REINTEGRADAS': [self.semanas_reintegradas],
            'SEMANAS TOTALES': [self.semanas_totales],
            'FECHA DE EMISIÓN DEL REPORTE': [self.fecha_emision_reporte]
        }
        )

        out.to_csv(name)
        return out

    def tabla_datos(self):
        out = pd.DataFrame({
            'NOMBRE': [self.Nombre],
            'NSS': [self.nss],
            'CURP': [self.curp],
            'FECHA DE NACIMIENTO': [self.fecha_nacimiento],
            'AÑO DE INSCRIPCIÓN IMSS': [self.AnoInicio],
            'REGIMEN AL QUE PERTENECE': [self.Regimen],
            'SEMANAS COTIZADAS': [self.semanas_cotizadas],
            'SEMANAS DESCONTADAS': [self.semanas_descontadas],
            'SEMANAS REINTEGRADAS': [self.semanas_reintegradas],
            'SEMANAS TOTALES': [self.semanas_totales],
            'FECHA DE EMISIÓN DEL REPORTE': [self.fecha_emision_reporte]
        }
        )
        out = out.T
        out.columns = ['DATOS PERSONALES']
        return out

# Fechas Generales

In [41]:
esempio = "13/10/1964"

In [6]:
class FechasGenerales:
    def __init__(self, texto, FechaEmisionReporte):
        self.texto = texto
        self.FechaEmisionReporte = FechaEmisionReporte

    def tabla_fechas_generales(self):
        texto = self.texto
        FechaEmisionReporte = self.FechaEmisionReporte
        # Define the search string
        bajas_string = 'Fecha de baja'
        value_added = 12
        # Find the indices of the search string
        bajas = [m.start() for m in re.finditer(bajas_string, texto)]
        # Get the index of the first occurrence
        ultima_baja = bajas[0]
        # Calculate the end index
        ultima_final = ultima_baja + len(bajas_string) + value_added
        # Extract the desired substring
        sigue_cotizando = "vigente" in texto.lower() #Check if sigue_cotizando is contained in the text
        # Initialize an array to store dates
        fecha_bajas = np.zeros(len(bajas), dtype=object)
        # Note: num2cell is not needed in Python, as numpy arrays can store objects

        for idx, baja in enumerate(bajas):
            if idx == 0:
                if not sigue_cotizando:
                    fechas_ultima_baja = datetime.strptime(texto[ultima_baja + len(bajas_string):ultima_final].strip(), "%d/%m/%Y")
                    fecha_bajas[idx] = fechas_ultima_baja
                else:
                    fecha_bajas[idx] = FechaEmisionReporte
                    fechas_ultima_baja = fecha_bajas[idx]

                if not isinstance(fechas_ultima_baja, date):
                    fechas_ultima_baja = fechas_ultima_baja.date()
            else:
                start_idx = baja + len(bajas_string)
                end_idx = start_idx + value_added  # Assuming 12 characters for the date
                fecha_bajas[idx] = texto[start_idx:end_idx].strip()
                fecha_bajas[idx] = datetime.strptime(fecha_bajas[idx], "%d/%m/%Y")


        altas_string = 'Fecha de alta'
        altas = [m.start() for m in re.finditer(altas_string, texto)]

        ultima_alta = altas[0]
        ultima_alta_final = ultima_alta + len(altas_string) + value_added

        fechas_ultima_alta = texto[ultima_alta + len(altas_string):ultima_alta_final].strip()

        fecha_altas = np.zeros(len(altas), dtype=object)
        dias_transcurridos_cotizados = np.zeros(len(altas), dtype=object)
        semanas_transcurridas_cotizadas = np.zeros(len(altas), dtype=object)

        for idx, alta in enumerate(altas):
            start_idx = alta + len(altas_string)
            end_idx = start_idx + value_added
            fecha_altas[idx] = datetime.strptime(texto[start_idx:end_idx].strip(), "%d/%m/%Y")
            # Convert fecha_bajas[idx] to datetime.datetime if it's a datetime.date
            if isinstance(fecha_bajas[idx], date):
                fecha_bajas[idx] = datetime.combine(fecha_bajas[idx], datetime.min.time())

            dias_transcurridos_cotizados[idx] = (fecha_bajas[idx] - fecha_altas[idx]).days
            semanas_transcurridas_cotizadas[idx] = dias_transcurridos_cotizados[idx] // 7

        inicio_patron_str = 'Nombre del patrón'
        final_patron_str = 'Registro Patronal'
        final_patron_str_entidad = 'Entidad federativa'
        inicio_patron = [m.start() for m in re.finditer(inicio_patron_str, texto)]
        final_patron = [m.start() for m in re.finditer(final_patron_str, texto)]

        if len(final_patron) != len(inicio_patron):
            missing_registro_patronal = [m.start() for m in re.finditer(final_patron_str_entidad, texto)]
            final_patron.append(missing_registro_patronal[-1])

        patrones = []
        # using list comprehension
        patrones = [texto[inicio + len(inicio_patron_str):final_patron[idx] - 1].strip()
                    for idx, inicio in enumerate(inicio_patron)]

        inicio_ef_str = 'Entidad federativa'
        final_ef_str = 'Fecha de alta'
        inicio_ef = [m.start() for m in re.finditer(inicio_ef_str, texto)]
        final_ef = [m.start() for m in re.finditer(final_ef_str, texto)]

        entidad_federativa = []

        for idx, inicio in enumerate(inicio_ef):
            start_idx = inicio + len(inicio_ef_str)
            end_idx = final_ef[idx] - 1
            entidad_federativa.append(texto[start_idx:end_idx].strip())

        # Alternatively, using list comprehension
        entidad_federativa = [texto[inicio + len(inicio_ef_str):final_ef[idx] - 1].strip()
                            for idx, inicio in enumerate(inicio_ef)]

        # Method using vectorize
        def format_date(date_str):
            try:
                # First try with time format
                date = datetime.strptime(str(date_str), '%Y-%m-%d %H:%M:%S')
            except ValueError:
                try:
                    # If that fails, try with just date
                    date = datetime.strptime(str(date_str), '%Y-%m-%d')
                except ValueError:
                    # If both fail, try to parse as datetime object directly
                    if isinstance(date_str, (datetime, date)):
                        date = date_str
                    else:
                        raise ValueError(f"Unsupported date format: {date_str}")
            return date.strftime('%d/%b/%Y')

        vectorized_format = np.vectorize(format_date)
        fecha_altas_str = vectorized_format(fecha_altas)
        fecha_bajas_str = vectorized_format(fecha_bajas)
        fechas_ultima_baja = format_date(fechas_ultima_baja)

        FechasGenerales = pd.DataFrame({
            'Fecha de alta': fecha_altas_str,
            'Fecha de baja': fecha_bajas_str,
            'Dias transcurridos cotizados': dias_transcurridos_cotizados,
            'Semanas transcurridas cotizadas': semanas_transcurridas_cotizadas,
            'Patrones': patrones,
            'Entidad federativa': entidad_federativa
        })
        return FechasGenerales, sigue_cotizando, fechas_ultima_baja

fechas_generales = FechasGenerales(texto=texto, FechaEmisionReporte=Usuario.fecha_emision_reporte)
fechasGenerales, sigueCotizando, fechasUltimaBaja = fechas_generales.tabla_fechas_generales()


In [1]:
class FechasGenerales:
    def __init__(self, texto, FechaEmisionReporte):
        self.texto = texto
        self.FechaEmisionReporte = FechaEmisionReporte

    def tabla_fechas_generales(self):
        texto = self.texto
        FechaEmisionReporte = self.FechaEmisionReporte
        # Define the search string
        bajas_string = 'Fecha de baja'
        value_added = 12
        # Find the indices of the search string
        bajas = [m.start() for m in re.finditer(bajas_string, texto)]
        # Get the index of the first occurrence
        ultima_baja = bajas[0]
        # Calculate the end index
        ultima_final = ultima_baja + len(bajas_string) + value_added
        # Extract the desired substring
        sigue_cotizando = "vigente" in texto.lower() #Check if sigue_cotizando is contained in the text
        # Initialize an array to store dates
        fecha_bajas = np.zeros(len(bajas), dtype=object)
        # Note: num2cell is not needed in Python, as numpy arrays can store objects

        for idx, baja in enumerate(bajas):
            if idx == 0:
                if not sigue_cotizando:
                    # fechas_ultima_baja = datetime.strptime(texto[ultima_baja + len(bajas_string):ultima_final].strip(), "%d/%m/%Y")
                    fechas_ultima_baja = texto[ultima_baja + len(bajas_string)
                    fecha_bajas[idx] = fechas_ultima_baja
                else:
                    fecha_bajas[idx] = FechaEmisionReporte
                    fechas_ultima_baja = fecha_bajas[idx]

                if not isinstance(fechas_ultima_baja, date):
                    fechas_ultima_baja = fechas_ultima_baja.date()
            else:
                start_idx = baja + len(bajas_string)
                end_idx = start_idx + value_added  # Assuming 12 characters for the date
                fecha_bajas[idx] = texto[start_idx:end_idx].strip()
                fecha_bajas[idx] = datetime.strptime(fecha_bajas[idx], "%d/%m/%Y")


        altas_string = 'Fecha de alta'
        altas = [m.start() for m in re.finditer(altas_string, texto)]

        ultima_alta = altas[0]
        ultima_alta_final = ultima_alta + len(altas_string) + value_added

        fechas_ultima_alta = texto[ultima_alta + len(altas_string):ultima_alta_final].strip()

        fecha_altas = np.zeros(len(altas), dtype=object)
        dias_transcurridos_cotizados = np.zeros(len(altas), dtype=object)
        semanas_transcurridas_cotizadas = np.zeros(len(altas), dtype=object)

        for idx, alta in enumerate(altas):
            start_idx = alta + len(altas_string)
            end_idx = start_idx + value_added
            fecha_altas[idx] = datetime.strptime(texto[start_idx:end_idx].strip(), "%d/%m/%Y")
            # Convert fecha_bajas[idx] to datetime.datetime if it's a datetime.date
            if isinstance(fecha_bajas[idx], date):
                fecha_bajas[idx] = datetime.combine(fecha_bajas[idx], datetime.min.time())

            dias_transcurridos_cotizados[idx] = (fecha_bajas[idx] - fecha_altas[idx]).days
            semanas_transcurridas_cotizadas[idx] = dias_transcurridos_cotizados[idx] // 7

        inicio_patron_str = 'Nombre del patrón'
        final_patron_str = 'Registro Patronal'
        final_patron_str_entidad = 'Entidad federativa'
        inicio_patron = [m.start() for m in re.finditer(inicio_patron_str, texto)]
        final_patron = [m.start() for m in re.finditer(final_patron_str, texto)]

        if len(final_patron) != len(inicio_patron):
            missing_registro_patronal = [m.start() for m in re.finditer(final_patron_str_entidad, texto)]
            final_patron.append(missing_registro_patronal[-1])

        patrones = []
        # using list comprehension
        patrones = [texto[inicio + len(inicio_patron_str):final_patron[idx] - 1].strip()
                    for idx, inicio in enumerate(inicio_patron)]

        inicio_ef_str = 'Entidad federativa'
        final_ef_str = 'Fecha de alta'
        inicio_ef = [m.start() for m in re.finditer(inicio_ef_str, texto)]
        final_ef = [m.start() for m in re.finditer(final_ef_str, texto)]

        entidad_federativa = []

        for idx, inicio in enumerate(inicio_ef):
            start_idx = inicio + len(inicio_ef_str)
            end_idx = final_ef[idx] - 1
            entidad_federativa.append(texto[start_idx:end_idx].strip())

        # Alternatively, using list comprehension
        entidad_federativa = [texto[inicio + len(inicio_ef_str):final_ef[idx] - 1].strip()
                            for idx, inicio in enumerate(inicio_ef)]

        # Method using vectorize
        def format_date(date_str):
            try:
                # First try with time format
                date = datetime.strptime(str(date_str), '%Y-%m-%d %H:%M:%S')
            except ValueError:
                try:
                    # If that fails, try with just date
                    date = datetime.strptime(str(date_str), '%Y-%m-%d')
                except ValueError:
                    # If both fail, try to parse as datetime object directly
                    if isinstance(date_str, (datetime, date)):
                        date = date_str
                    else:
                        raise ValueError(f"Unsupported date format: {date_str}")
            return date.strftime('%d/%b/%Y')

        vectorized_format = np.vectorize(format_date)
        fecha_altas_str = vectorized_format(fecha_altas)
        fecha_bajas_str = vectorized_format(fecha_bajas)
        fechas_ultima_baja = convertir_a_fecha(fechas_ultima_baja)

        FechasGenerales = pd.DataFrame({
            'Fecha de alta': fecha_altas_str,
            'Fecha de baja': fecha_bajas_str,
            'Dias transcurridos cotizados': dias_transcurridos_cotizados,
            'Semanas transcurridas cotizadas': semanas_transcurridas_cotizadas,
            'Patrones': patrones,
            'Entidad federativa': entidad_federativa
        })
        return FechasGenerales, sigue_cotizando, fechas_ultima_baja

fechas_generales = FechasGenerales(texto=texto, FechaEmisionReporte=Usuario.fecha_emision_reporte)
fechasGenerales, sigueCotizando, fechasUltimaBaja = fechas_generales.tabla_fechas_generales()


SyntaxError: '[' was never closed (2801733296.py, line 28)

In [8]:
sigueCotizando

True

In [9]:
fechasGenerales

,Fecha de alta,Fecha de baja,Dias transcurridos cotizados,Semanas transcurridas cotizadas,Patrones,Entidad federativa
0,01/Sep/2022,07/Sep/2024,737,105,CONTINUACION VOLUNTARIA EN EL REGIMEN OBLIGATORIO,CHIAPAS
1,05/Oct/1989,31/Aug/2022,12018,1716,COLEGIO MARIANO N RUIZ,CHIAPAS


In [10]:
def fecha_pension_minima(fecha_nacimiento, fechasUltimaBaja):
    if isinstance(fecha_nacimiento, str):
        fecha_nacimiento = datetime.strptime(fecha_nacimiento, '%b/%d/%Y')
    elif isinstance(fechasUltimaBaja, str):
        fechasUltimaBaja = datetime.strptime(fechasUltimaBaja, '%d/%b/%Y').date()

    # edad_actual = (datetime.today().date() - fecha_nacimiento.date()).days // 365
    edad_actual = round((datetime.today().date() - fecha_nacimiento).days / 365, 2)
    fecha_60 = fecha_nacimiento + relativedelta(years=60)
    print(f"Edad al dia de hoy: {edad_actual}")
    print(f"Fecha en que cumple 60 años: {fecha_60}")
    print(f"FECHA DE ULTIMA BAJA: {fechasUltimaBaja}")
    fecha_pension_minima = max(fecha_60, fechasUltimaBaja)
    return fecha_pension_minima

fecha_pension_minima = fecha_pension_minima(fecha_nacimiento=Usuario.fecha_nacimiento, fechasUltimaBaja=fechasUltimaBaja)
fecha_pension_minima

Edad al dia de hoy: 60.86
Fecha en que cumple 60 años: 2024-10-13
FECHA DE ULTIMA BAJA: 2024-09-07


datetime.date(2024, 10, 13)

# Historial Laboral Desglosado

In [11]:
def HistorialLaboralTabla(texto):
    # Define the search string
    BloqueInicio = 'Nombre del patrón'
    BloqueFinal = '* Valor del último salario base de cotización diario en pesos.'

    # Find occurrences of the start and end blocks
    Inicio = [m.start() for m in re.finditer(re.escape(BloqueInicio), texto)]
    print(Inicio)
    Final = [m.start() for m in re.finditer(re.escape(BloqueFinal), texto)]
    # print(Final, BloqueFinal in texto)
    print(texto)
    HistoriaLaboral = []

    # Extract information between blocks
    for start, end in zip(Inicio, Final):
        substring = texto[start + len(BloqueInicio):end].strip()
        output = [line.strip() for line in substring.splitlines() if len(line.strip()) > 1]
        HistoriaLaboral.append(output)

    # Initialize an empty DataFrame for the final table
    HistoriaLaboralTable = pd.DataFrame()
    # target_strings = {'ALTA', 'REINGRESO', 'MODIFICACION DE SALARIO', 'BAJA'}
    target_strings = {'ALTA', 'REINGRESO', 'MODIFICACION', 'BAJA'}
    Movimiento = list()
    FechaMovimiento = list()
    Sueldo = list()
    Empleador = list()
    EntidadFederativa = list()
    for entry in HistoriaLaboral:
        for line in entry:
            # if any(string in line for string in target_strings):
            if any(string in line.split()[0] for string in target_strings):
                Empleador.append(entry[0])
                EntidadFederativa.append(entry[2].split()[2:])
                array = line.split()
                if array[0] == 'MODIFICACION':
                    Movimiento.append('MODIFICACION DE SALARIO')
                else:
                    Movimiento.append(array[0])
                FechaMovimiento.append(array[-3])
                Sueldo.append(array[-1])

    # Convert 'Sueldo' elements to float before formatting
    Sueldo_formatted = [float(s) for s in Sueldo]
    HistoriaLaboralTable_str = pd.DataFrame({
        'Movimiento': Movimiento,
        'Fecha de Movimiento': FechaMovimiento,
        'Sueldo': Sueldo, #np.vectorize(lambda x: "${:,.2f}".format(x))(Sueldo_formatted),
        'Empleador': Empleador,
        'Entidad Federativa': EntidadFederativa
    })

    HistoriaLaboralTable_str['Entidad Federativa'] = HistoriaLaboralTable_str['Entidad Federativa'].apply(lambda x: " ".join(x))

    HistoriaLaboralTable_num = HistoriaLaboralTable_str.copy()
    HistoriaLaboralTable_num['Sueldo'] = Sueldo_formatted
    return HistoriaLaboralTable_str, HistoriaLaboralTable_num

HistoriaLaboralTable_str, HistoriaLaboralTable_num = HistorialLaboralTabla(texto)
def HistorialLaboralDesglosada_fcn(texto, sigueCotizando, FechasUltimaBaja):
    # Define the search string
    HistoriaLaboralTable, _ = HistorialLaboralTabla(texto)

    from datetime import timedelta
    Bajas = np.where(HistoriaLaboralTable['Movimiento'] == 'BAJA')[0]
    FechaFinal = HistoriaLaboralTable['Fecha de Movimiento'].values

    for idx in range(len(HistoriaLaboralTable)):
        FechaFinal[idx] = datetime.strptime(FechaFinal[idx], '%d/%m/%Y').date()
        if idx not in Bajas:
            FechaFinal[idx] =  FechaFinal[idx] - timedelta(days=1)


    FechaFinal = np.roll(FechaFinal, 1)

    if sigueCotizando:
        FechaFinal[0] = FechasUltimaBaja

    HistoriaLaboralDesglosada = HistoriaLaboralTable.copy()
    HistoriaLaboralDesglosada['Fecha Final'] = FechaFinal
    HistoriaLaboralDesglosada.drop(Bajas, inplace=True)
    HistoriaLaboralDesglosada = HistoriaLaboralDesglosada.rename(columns={"Fecha de Movimiento": "Fecha Inicial"})

    HistoriaLaboralDesglosada["Fecha Inicial"] = pd.to_datetime(HistoriaLaboralDesglosada["Fecha Inicial"], format='%d/%m/%Y')

    HistoriaLaboralDesglosada["Fecha Inicial"] = HistoriaLaboralDesglosada["Fecha Inicial"] + timedelta(days = 1)
    # Move the column "FechaFinal" to be after "FechaInicial"
    cols = list(HistoriaLaboralDesglosada.columns)
    cols.insert(cols.index("Fecha Inicial") + 1, cols.pop(cols.index("Fecha Final")))
    HistoriaLaboralDesglosada = HistoriaLaboralDesglosada[cols]
    # We will change the date format for the column "Fecha Inicial"
    HistoriaLaboralDesglosada['Fecha Inicial'] = pd.to_datetime(HistoriaLaboralDesglosada['Fecha Inicial'], format='%d/%m/%Y').dt.date
    return HistoriaLaboralDesglosada

HistoriaLaboralTable = HistorialLaboralDesglosada_fcn(texto, sigueCotizando, fechasUltimaBaja)

print("Historial Laboral:")
HistoriaLaboralTable_str.head()

[416, 772]
Instituto Mexicano del Seguro Social
Constancia de Semanas Cotizadas en el IMSS
Estimado(a), Fecha de emisión del reporte
07 / 09 / 2024
GORDILLO AVENDAÑO SARA EUGENIA
DD MM YYYY
NSS: 71896435444 Total de semanas cotizadas
CURP: GOAS641013MCSRVR01 1822
Tu detalle de semanas cotizadas
Semanas Descontadas
Semanas cotizadas IMSS Semanas Reintegradas (+)
(por disposición de recursos) (-)
1822 0 0
Tu historia laboral
Nombre del patrón CONTINUACION VOLUNTARIA EN EL REGIMEN OBLIGATORIO
Registro Patronal A689999940
Entidad federativa CHIAPAS
Fecha de alta 01/09/2022 Fecha de baja Vigente Salario Base de Cotización */ $ 2405.5
Tipo de movimiento Fecha de movimiento Salario Base
REINGRESO 01/09/2022 $ 2405.5
/* Valor del último salario base de cotización diario en pesos.
Nombre del patrón COLEGIO MARIANO N RUIZ
Registro Patronal A541074010
Entidad federativa CHIAPAS
Fecha de alta 05/10/1989 Fecha de baja 31/08/2022 Salario Base de Cotización */ $ 409.35
Tipo de movimiento Fecha de mov

,Movimiento,Fecha de Movimiento,Sueldo,Empleador,Entidad Federativa
0,REINGRESO,01/09/2022,2405.5,CONTINUACION VOLUNTARIA EN EL REGIMEN OBLIGATORIO,CHIAPAS
1,BAJA,31/08/2022,409.35,COLEGIO MARIANO N RUIZ,CHIAPAS
2,MODIFICACION DE SALARIO,01/01/2022,409.35,COLEGIO MARIANO N RUIZ,CHIAPAS
3,MODIFICACION DE SALARIO,01/01/2021,371.66,COLEGIO MARIANO N RUIZ,CHIAPAS
4,MODIFICACION DE SALARIO,01/01/2020,337.87,COLEGIO MARIANO N RUIZ,CHIAPAS


# Tabla de Salarios

In [12]:
def salario_promedio_250tabla(HistoriaLaboralDesglosada):
    salario_promedio_250 = HistoriaLaboralDesglosada[["Fecha Inicial", "Fecha Final", "Sueldo"]]
    salario_promedio_250 = salario_promedio_250.sort_values(by=["Fecha Inicial", "Fecha Final"], ascending=[False, False])
    # We will change the date format for the first column
    salario_promedio_250['Fecha Inicial'] = pd.to_datetime(salario_promedio_250['Fecha Inicial'], format='%d/%m/%Y').dt.date
    return salario_promedio_250

tabla_salarios = salario_promedio_250tabla(HistoriaLaboralTable)
tabla_salarios.head()

,Fecha Inicial,Fecha Final,Sueldo
0,2022-09-01,07/Sep/2024,2405.5
2,2022-01-01,2022-08-31,409.35
3,2021-01-01,2021-12-31,371.66
4,2020-01-01,2020-12-31,337.87
5,2019-01-01,2019-12-31,307.16


In [13]:
def tabla_salarios_modificada(tabla_salarios_original,fecha_inicio, fecha_final, salario):
    display(tabla_salarios_original)
    nueva_fila = {'Fecha Inicial': fecha_inicio, 'Fecha Final': fecha_final, 'Sueldo': salario}
    nueva_tabla = pd.concat([tabla_salarios_original, pd.DataFrame([nueva_fila])], ignore_index=True)
    # We will sort by the column "Fecha Inicial"
    nueva_tabla = nueva_tabla.sort_values(by=["Fecha Inicial"], ascending=[False])
    # We will change the date format for the first column
    nueva_tabla['Fecha Inicial'] = pd.to_datetime(nueva_tabla['Fecha Inicial'], format='%d/%m/%Y').dt.date
    # we will change the date format for the second column, from day/month/year to year-month-day
    nueva_tabla['Fecha Final'] = pd.to_datetime(nueva_tabla['Fecha Final'], format='%d/%b/%Y').dt.date
    display(nueva_tabla)
    return nueva_tabla

fecha_inicio = pd.to_datetime('2023-01-01').date()
fecha_final = pd.to_datetime('2023-12-11').date()
salario = 1000

nueva_tabla_salarios = tabla_salarios_modificada(tabla_salarios_original=tabla_salarios,
                          fecha_inicio=fecha_inicio,
                          fecha_final=fecha_final,
                          salario=salario)

,Fecha Inicial,Fecha Final,Sueldo
0,2022-09-01,07/Sep/2024,2405.5
2,2022-01-01,2022-08-31,409.35
3,2021-01-01,2021-12-31,371.66
4,2020-01-01,2020-12-31,337.87
5,2019-01-01,2019-12-31,307.16
6,2018-01-01,2018-12-31,267.09
7,2017-01-01,2017-12-31,242.8
8,2016-01-01,2016-12-31,211.13
9,2015-01-01,2015-12-31,191.94
10,2014-01-01,2014-12-31,182.8


,Fecha Inicial,Fecha Final,Sueldo
29,2023-01-01,2023-12-11,1000
0,2022-09-01,2024-09-07,2405.5
1,2022-01-01,2022-08-31,409.35
2,2021-01-01,2021-12-31,371.66
3,2020-01-01,2020-12-31,337.87
4,2019-01-01,2019-12-31,307.16
5,2018-01-01,2018-12-31,267.09
6,2017-01-01,2017-12-31,242.8
7,2016-01-01,2016-12-31,211.13
8,2015-01-01,2015-12-31,191.94


## Tabla de salarios promedios para el cálculo de las últimas 250 semanas

In [14]:
def salario_promedio_fcn(semanas_contar, semanas_reconocidas, salario_promedio_250):
    if semanas_contar == 0:
        return 0, pd.DataFrame()
  # Reset the index of salario_promedio_250 to ensure it starts from 0
    salario_promedio_250 = salario_promedio_250.reset_index(drop=True)
    # Convert Fecha Inicial and Fecha Final columns to datetime.date
    salario_promedio_250['Fecha Inicial'] = pd.to_datetime(salario_promedio_250['Fecha Inicial']).dt.date
    salario_promedio_250['Fecha Final'] = pd.to_datetime(salario_promedio_250['Fecha Final']).dt.date
    # Initialize necessary variables
    salario_acumulado_periodo = np.zeros(len(salario_promedio_250['Fecha Inicial']))
    fecha_superior = salario_promedio_250['Fecha Final']
    fecha_inferior = salario_promedio_250['Fecha Inicial']
    salario_diario =  np.array(salario_promedio_250['Sueldo']).astype(float)
    semanas_periodo = salario_acumulado_periodo.copy()
    fecha_corte = [None] * len(fecha_superior)
    semanas_acumuladas = salario_acumulado_periodo.copy()
    semanas_cuenta = salario_acumulado_periodo.copy()

    for idx in range(len(fecha_superior)):
        # Calculate weeks in each period
        semanas_periodo[idx] = (fecha_superior[idx] - fecha_inferior[idx]).days / 7 + 1 / 7
        if idx == 0:
            if semanas_periodo[idx] <= semanas_contar:
                fecha_corte[idx] = fecha_inferior[idx]
            else:
                fecha_corte[idx] = fecha_superior[idx] - timedelta(weeks=semanas_contar)

            semanas_cuenta[idx] = (fecha_superior[idx] - fecha_inferior[idx]).days / 7 + 1 / 7
            semanas_acumuladas[idx] = semanas_periodo[idx]
        else:
            semanas_acumuladas[idx] = semanas_periodo[idx]

            if semanas_acumuladas[idx - 1] < semanas_contar:
                if semanas_acumuladas[idx - 1] + semanas_periodo[idx] < semanas_contar:
                    if fecha_corte[idx - 1] is None:  # Check if previous fecha_corte is None
                        fecha_corte[idx] = fecha_inferior[idx]  # Use fecha_inferior if previous is None
                    else:
                        fecha_corte[idx] = min(fecha_corte[idx - 1], fecha_inferior[idx])  # Use min if previous is not None

                else:
                    fecha_corte[idx] = fecha_superior[idx] - timedelta(weeks=(semanas_contar - semanas_acumuladas[idx - 1]))
            else:
                fecha_corte[idx] = fecha_corte[idx - 1]

            if fecha_corte[idx] > fecha_superior[idx]:
                semanas_cuenta[idx] = 0
            else:
                semanas_cuenta[idx] = (fecha_superior[idx] - max(fecha_corte[idx], fecha_inferior[idx])).days / 7 + 1 / 7

            if semanas_acumuladas[idx - 1] == semanas_contar:
                semanas_acumuladas[idx] = semanas_contar
            else:
                if semanas_acumuladas[idx - 1] + semanas_cuenta[idx] > semanas_contar:
                    semanas_acumuladas[idx] = semanas_contar
                else:
                    if fecha_corte[idx] != fecha_inferior[idx]:
                        semanas_acumuladas[idx] = semanas_acumuladas[idx - 1] + (max(fecha_corte[idx - 1], fecha_superior[idx]) - fecha_corte[idx]).days / 7 + 1 / 7
                    else:
                        semanas_acumuladas[idx] = semanas_acumuladas[idx - 1] + (min(fecha_corte[idx - 1], fecha_superior[idx]) - fecha_corte[idx]).days / 7 + 1 / 7

        salario_acumulado_periodo[idx] = salario_diario[idx] * 7 * semanas_cuenta[idx] if semanas_cuenta[idx] > 0 else 0

    tabla_salario_promedio = pd.DataFrame({
        'Fecha Inicio': fecha_inferior,
        'Fecha Final': fecha_superior,
        # Convert to string with currency format
        'Salario Diario': np.vectorize(lambda x: "${:,.2f}".format(x))(salario_diario),
        'Semanas totales en el periodo': np.round(semanas_periodo, 1),
        'Fecha de Corte': pd.to_datetime(fecha_corte),
        'Semanas tomadas en cuenta': np.round(semanas_cuenta, 1),
        'Salario Acumulado en el Periodo': np.vectorize(lambda x: "${:,.2f}".format(x))(np.round(salario_acumulado_periodo)),
        'Semanas Acumuladas Totales': np.round(semanas_acumuladas, 1)
    })
    tabla_salario_promedio['Fecha de Corte'] = tabla_salario_promedio['Fecha de Corte'].dt.date
    salario_acumulado = np.sum(salario_acumulado_periodo)

    if semanas_reconocidas < semanas_contar:
        salario_promedio_diario = salario_acumulado / (semanas_reconocidas * 7)
    else:
        salario_promedio_diario = salario_acumulado / (semanas_contar * 7)

    return salario_promedio_diario, tabla_salario_promedio

SEMANAS_CONTAR = 250
SemanasReconocidas = Usuario.semanas_totales
SALARIO_PROMEDIO_DIARIO, TABLA_SALARIO_PROMEDIO = salario_promedio_fcn(SEMANAS_CONTAR, SemanasReconocidas, tabla_salarios)
display(TABLA_SALARIO_PROMEDIO.head())
print(f"Salario Promedio:\n{round(SALARIO_PROMEDIO_DIARIO, 1)}")

,Fecha Inicio,Fecha Final,Salario Diario,Semanas totales en el periodo,Fecha de Corte,Semanas tomadas en cuenta,Salario Acumulado en el Periodo,Semanas Acumuladas Totales
0,2022-09-01,2024-09-07,"$2,405.50",105.4,2022-09-01,105.4,"$1,775,259.00",105.4
1,2022-01-01,2022-08-31,$409.35,34.7,2022-01-01,34.7,"$99,472.00",140.1
2,2021-01-01,2021-12-31,$371.66,52.1,2021-01-01,52.1,"$135,656.00",192.3
3,2020-01-01,2020-12-31,$337.87,52.3,2020-01-01,52.3,"$123,660.00",244.6
4,2019-01-01,2019-12-31,$307.16,52.1,2019-11-23,5.6,"$11,979.00",250.0


Salario Promedio:
1226.3


In [15]:
# Cálculo de Obtención de Pensión
edad_pension = 60 #@param {type:"integer"}
year = 2022 #@param {type:"integer"}
semanas_reconocidas = 966 #@param {type:"integer"}
salario_promedio = 1439.16 #@param {type:"number"}
porcentaje_asignaciones = 0.15 #@param {type:"number"}

class PensionManual:
    def __init__(self, edad_pension, year, semanas_reconocidas, salario_promedio, porcentaje_asignaciones):
        self.edad_pension = edad_pension
        self.year = year
        self.semanas_reconocidas = semanas_reconocidas
        self.salario_promedio = salario_promedio
        self.porcentaje_asignaciones = porcentaje_asignaciones
        self.tabla_salarios_minimos = pd.read_csv('TABLAS/TABLA_SALARIOSMINIMOS.txt', sep='\t')
        self.tabla_porcentaje_edad = pd.read_csv('TABLAS/TABLA_PORCENTAJE_EDAD.txt', sep='\t')
        self.tabla_uma = pd.read_csv('TABLAS/TABLA_UMA.txt', sep='\t')
        self.tabla_relacion_umas = pd.read_csv('TABLAS/TABLA_RELACION_UMAS_SALARIO_MINIMO.txt', sep='\t')

    def salario_minimo_garantizado_fcn(self):
            tabla = self.tabla_salarios_minimos
            out = tabla[tabla.iloc[:, 0] == year]
            salario_minimo_ano_actual = out.iloc[0, 1].strip("$")
            salario_minimo_mensual = out.iloc[0, 4]
            pension_minima_garantizada = out.iloc[0, -1]
            salario_maximo_asignado_diario = out.iloc[0, 6]

            print(f"Los siguientes datos son para el año: {self.year}")

            # We will also create a pandas dataframe
            df = pd.DataFrame({
                # 'Año': year,
                'Salario Minimo Diario': salario_minimo_ano_actual,
                'Salario Minimo Mensual': salario_minimo_mensual,
                'Pension Mínima Garantizada': pension_minima_garantizada,
                'Salario Máximo Asignado Diario': salario_maximo_asignado_diario
            }, index=[self.year])
            display(df)

            self.salario_minimo_ano_actual = salario_minimo_ano_actual
            self.salario_minimo_mensual = salario_minimo_mensual
            self.pension_minima_garantizada = pension_minima_garantizada
            self.salario_maximo_asignado_diario = salario_maximo_asignado_diario

            return salario_minimo_ano_actual, salario_minimo_mensual, pension_minima_garantizada, salario_maximo_asignado_diario, df

    def porcentaje_vejez_fcn(self):
        tabla = self.tabla_porcentaje_edad
        idx = tabla[tabla['EDAD'] == min(self.edad_pension, 65)]['% PENSIÓN'] # Finds the position of the table with the target age
        perc_string = idx.iloc[0] # Obtain the percentage by multiplying by 100%
        porcentaje_vejez =float( perc_string.strip('%')) / 1e2
        self.porcentaje_vejez = porcentaje_vejez

    def valor_uma_actual_year_fcn(self):
        tabla = self.tabla_uma
        idx = tabla[tabla['AÑO'] == self.year]['VALOR $'] # Finds the position of the table with the target year
        valor_uma_string = idx.iloc[0]
        valor_uma_numerico =float( valor_uma_string.strip('$'))
        self.valor_uma_numerico = valor_uma_numerico

    def relacion_uma_salario_fcn(self):
        relacion_salario_uma = round(self.salario_promedio / self.valor_uma_numerico, 2)
        self.relacion_uma_salario = relacion_salario_uma

    def porcentaje_cuantia_fcn(self):
        tabla = self.tabla_relacion_umas
        #subs substracts the variable from all values and then we take only those values inferior, finally we take the pos. of the largest value
        closest_idx = tabla[tabla["Inferior"].sub(self.relacion_uma_salario) < 0]['Inferior'].idxmax()
        out_string = tabla.iloc[closest_idx, 2] # Pos. 2 relates to Porcentaje cuantía básica [%]
        porcentaje_cuantia = float(out_string.strip(('%')))/1e2
        self.porcentaje_cuantia = porcentaje_cuantia

    def semanas_equivalentes_year_fcn(self):
        semanas_despues500 = semanas_reconocidas - 500
        upper_bound = 0.5
        lower_bound = 0.25
        if lower_bound < abs(semanas_despues500/52 - np.fix(semanas_despues500/52)) <= upper_bound:
            SEMANAS_EQUIVALENTES = np.fix(semanas_despues500 / 52) + 0.5
        else:
            SEMANAS_EQUIVALENTES = np.round(semanas_despues500/52, decimals=0)
        self.semanas_equivalentes = SEMANAS_EQUIVALENTES

    def salario_incremento_anual_perc_fcn(self):
        tabla = self.tabla_relacion_umas
        closest_idx = (tabla['Porcentaje cuantía básica \r\n[%]'].str.replace('%', '', regex=True)).astype(float)
        closest_idx = closest_idx.sub(self.porcentaje_cuantia*1e2) <= 0
        closest_idx = tabla[closest_idx]['Salario incremento anual [%]'].idxmax()
        salario_incremento_anual_perc = tabla.loc[closest_idx, 'Salario incremento anual [%]']
        salario_incremento_anual_perc = float(salario_incremento_anual_perc.strip('%')) / 1e2
        self.salario_incremento_anual_perc = salario_incremento_anual_perc

    def incremento_perc_fcn(self):
        incremento_perc = round(self.salario_incremento_anual_perc * self.semanas_equivalentes, 4)
        self.incremento_perc = incremento_perc
        return incremento_perc

    def incremento_dinero_fcn(self):
        incremento_dinero = round(self.incremento_perc * self.salario_promedio * 1.11 * 365, 2)
        self.incremento_dinero = incremento_dinero
        # return incremento_dinero

    def cuantia_basica_anual_fcn(self):
        cuantia_basica_anual = round(self.porcentaje_cuantia * self.salario_promedio * 365 * 1.11, 2)
        self.cuantia_basica_anual = cuantia_basica_anual
        # return cuantia_basica_anual

    def cuantia_vejez_fcn(self):
        cuantia_vejez = self.cuantia_basica_anual + self.incremento_dinero
        self.cuantia_vejez = cuantia_vejez

    def pension_cesantia_edad_avanzada_fcn(self):
        pension_cesantia_edad_avanzada = round(self.cuantia_vejez * self.porcentaje_vejez, 2)
        self.pension_cesantia_edad_avanzada = pension_cesantia_edad_avanzada

    def importe_asignaciones_familiares_fcn(self):
        importe_asignaciones_familiares = round(self.porcentaje_asignaciones * self.pension_cesantia_edad_avanzada, 2)
        self.importe_asignaciones_familiares = importe_asignaciones_familiares

    def importe_anual_cesantia_fcn(self):
        importe_anual_cesantia = round(self.importe_asignaciones_familiares + self.pension_cesantia_edad_avanzada, 2)
        self.importe_anual_cesantia = importe_anual_cesantia

    def importe_mensual_calculado_fcn(self):
        importe_mensual_calculado = round(self.importe_anual_cesantia / 12, 1)
        self.importe_mensual_calculado = importe_mensual_calculado

    def pension_minima_garantizada_fcn(self):
        pension_minima_garantizada = self.pension_minima_garantizada.replace('$', '').replace(',', '')
        pension_minima_garantizada = float(pension_minima_garantizada)
        self.pension_minima_garantizada = pension_minima_garantizada

    def pension_final_fcn(self):
        pension_final = round(max(self.importe_mensual_calculado, self.pension_minima_garantizada), 2)
        self.pension_final = pension_final

    def calculo_pension(self):
        self.salario_minimo_garantizado_fcn()
        self.porcentaje_vejez_fcn()
        self.valor_uma_actual_year_fcn()
        self.relacion_uma_salario_fcn()
        self.porcentaje_cuantia_fcn()
        self.semanas_equivalentes_year_fcn()
        self.salario_incremento_anual_perc_fcn()
        self.incremento_perc_fcn()
        self.incremento_dinero_fcn()
        self.cuantia_basica_anual_fcn()
        self.cuantia_vejez_fcn()
        self.pension_cesantia_edad_avanzada_fcn()
        self.importe_asignaciones_familiares_fcn()
        self.importe_anual_cesantia_fcn()
        self.importe_mensual_calculado_fcn()
        self.pension_minima_garantizada_fcn()
        self.pension_final_fcn()

        pension_df = pd.DataFrame({'Pensión Final': [self.pension_final],
                                   'Semanas Reconocidas': [self.semanas_reconocidas],
                                   'Salario Promedio': [self.salario_promedio],
                                   'Pension Final': [self.pension_final]
                                   })

        detalles_df = pd.DataFrame({
                                   'Porcentaje Asignaciones Familiares': [self.porcentaje_asignaciones],
                                   'Porcentaje Vejez': [self.porcentaje_vejez],
                                   'Valor UMA Actual Year': [self.valor_uma_numerico],
                                   'Incremento %': [self.incremento_perc],
                                   'Incremento Dinero': [self.incremento_dinero],
                                   'Cuantia Basica Anual': [self.cuantia_basica_anual],
                                   'Cuantia Vejez': [self.cuantia_vejez],
                                   'Pension Cesantia Edad Avanzada': [self.pension_cesantia_edad_avanzada],
                                   'Importe Asignaciones Familiares': [self.importe_asignaciones_familiares],
                                   'Importe Anual Cesantia': [self.importe_anual_cesantia],
                                   'Importe Mensual Calculado': [self.importe_mensual_calculado],
                                   'Pension Minima Garantizada': [self.pension_minima_garantizada],
                                   'Pension Final': [self.pension_final]
                                   })

        detalles_df = round(detalles_df.T, 2)
        # We will rename the column
        detalles_df.columns = ['Valores']

        return pension_df, detalles_df

# Example Usage
pension_manual = PensionManual(edad_pension=edad_pension, year=year, semanas_reconocidas=semanas_reconocidas, salario_promedio=salario_promedio, porcentaje_asignaciones=porcentaje_asignaciones)


pension_df, detalles_df = pension_manual.calculo_pension()
display(pension_df)
display(detalles_df)

Los siguientes datos son para el año: 2022


,Salario Minimo Diario,Salario Minimo Mensual,Pension Mínima Garantizada,Salario Máximo Asignado Diario
2022,172.87,"$5,258.13","$5,836.52","$2,405.50"


,Pensión Final,Semanas Reconocidas,Salario Promedio,Pension Final
0,14689.0,966,1439.16,14689.0


,Valores
Porcentaje Asignaciones Familiares,0.15
Porcentaje Vejez,0.75
Valor UMA Actual Year,96.22
Incremento %,0.22
Incremento Dinero,128568.19
Cuantia Basica Anual,75799.84
Cuantia Vejez,204368.03
Pension Cesantia Edad Avanzada,153276.02
Importe Asignaciones Familiares,22991.40
Importe Anual Cesantia,176267.42


In [16]:
def tabla_aguinaldo(fecha_inicio, pension_final, year_max, perc_asignaciones_familiares, year_cambio_asignaciones = None, perc_restar = None):
    tabla = 'TABLAS/TABLA_INFLACION.txt'
    tabla = pd.read_csv(tabla, sep = '\t')
    # Convert fecha_inicio to datetime.date if it's datetime.datetime
    if isinstance(fecha_inicio, datetime):
        fecha_inicio = fecha_inicio.date()

    year_pension = fecha_inicio.year
    num_years = year_max - year_pension
    idx_inicial = tabla.index[tabla['AÑO'] == year_pension].tolist()[0]

    idx_final = tabla.index[tabla['AÑO'] == year_pension + num_years].tolist()[0]
    inflacion_anual_numerico = tabla.iloc[idx_inicial:idx_final, 1]
    inflacion_anual_numerico = inflacion_anual_numerico.map(lambda x: float(x.replace('%','')))
    inflacion_anual_acumulada = np.cumsum(inflacion_anual_numerico)
    inflacion_anual_acumulada = np.insert(inflacion_anual_acumulada, 0, 0)

    year_max = min(2040, year_max)
    fecha_final = date(year_max, 12, 1)

    year_array = np.arange(0, num_years + 1) + year_pension
    asignaciones_array = np.full(len(year_array), perc_asignaciones_familiares)


    pension_year_final = date(year_pension, 12, 31)

    days_rd = (pension_year_final - fecha_inicio).days

    fraction_aguinaldo = days_rd / 365
    aguinaldo_array = list()
    aguinaldo_array.append(fraction_aguinaldo * (1 - perc_asignaciones_familiares) * pension_final)
    if year_cambio_asignaciones is not None:
        year_reduccion, perc_asignaciones_familiares_nuevas = year_cambio_asignaciones, perc_restar
    for idx in range(len(year_array) - 1):
        if year_reduccion:
            if year_array[idx + 1] == year_reduccion:
                asignaciones_array[idx + 1:] = max(perc_asignaciones_familiares - perc_asignaciones_familiares_nuevas, 0.15)
            aguinaldo_array.append((1 - asignaciones_array[idx + 1]) * pension_final * (1 + inflacion_anual_acumulada[idx + 1] / 100))

    risultati_num = pd.DataFrame({'Año' : year_array,
                              'Aguinaldo' : pd.Series(aguinaldo_array),
                              '% de Asignaciones Familiares': asignaciones_array
                              })
    risultati_num = risultati_num.round(2)

    risultati_str = pd.DataFrame({'Año' : year_array,
                              'Aguinaldo' : pd.Series(aguinaldo_array).apply(lambda x: "${:,.2f}".format(x)),
                              '% de Asignaciones Familiares': pd.Series(asignaciones_array).apply(lambda x: "{:.0f}%".format(x*1e2))
                              })

    return risultati_num, risultati_str

# Fechas de Pago Modalidad 40

In [17]:
def pagos40(salario, start_date, durata):
    tabla_uma = pd.read_csv('TABLAS/TABLA_UMA.txt', sep = '\t')
    TABLA_AUMENTO_PAGOMOD40 = pd.read_csv('TABLAS/TABLA_AUMENTOPAGO40.txt', sep = '\t')
    TABLA_SALARIOSMINIMOS = pd.read_csv('TABLAS/TABLA_SALARIOSMINIMOS.txt', sep = '\t')
    CUOTA_OBRERA = 6.925 /1e2;

    salario_array = np.full(durata + 1, salario) # Creates an array with the same value
    second_date = start_date + relativedelta(months = 1)
    second_date = second_date.replace(day = 1)

    monthly_series = pd.date_range(start=second_date, periods= durata, freq='MS') # MS: Month Start
    time_series_df = pd.concat([ pd.Series(start_date), pd.Series(monthly_series)])
    time_series_df = pd.to_datetime(time_series_df)

    salarios_minimos_array = list()
    salarios_maximos_array = list()
    salario_uma40_array = list()
    DiasCubiertosMensuales_array = list()
    PORCENTAJE_PAGO_array = list()

    def money_to_float(input_str: str, *args):
        for chars in args:
            input_str = input_str.replace(chars, '')
        return float(input_str.strip())
    # Wtih applymap we apply our custom function to each element specified in the iloc
    # indexation, we remove the '%' with the replace function and divide by 1e2
    TABLA_AUMENTO_PAGOMOD40.iloc[:, 1:] = TABLA_AUMENTO_PAGOMOD40.iloc[:, 1:].map(lambda x: money_to_float(x, '%')/1e2)
    TABLA_AUMENTO_PAGOMOD40 = TABLA_AUMENTO_PAGOMOD40.set_index('AÑO')
    salarios_minimos = TABLA_SALARIOSMINIMOS.loc[:, 'SM DIARIO ($)'].map(lambda x: money_to_float(x, '$'))
    salarios_maximos = TABLA_SALARIOSMINIMOS.iloc[:, 6].map(lambda x: money_to_float(x, '$', ','))

    for i in range(durata + 1):
        if time_series_df.iloc[i].month == 1:
            year_considered = time_series_df.iloc[i].year - 1
        else:
            year_considered = time_series_df.iloc[i].year
        # Find the index of rows where the 'AÑO' column equals 'year_considered'

        row = tabla_uma[tabla_uma['AÑO'] == year_considered].index
        row = row.tolist()[0]
        perc_row = tabla_uma.index[tabla_uma['AÑO'] == time_series_df.iloc[i].year].tolist()[0]
        salarios_minimos_array.append(salarios_minimos[tabla_uma['AÑO'] == time_series_df.iloc[i].year].tolist()[0])
        salarios_maximos_array.append(salarios_maximos[tabla_uma['AÑO'] == time_series_df.iloc[i].year].tolist()[0])
        salario_array[i] = min(salarios_maximos_array[i], max(salarios_minimos_array[i], salario_array[i]))

        uma_row = tabla_uma.iloc[perc_row, 1]
        uma_row = uma_row.replace('$', '')
        uma_row = float(uma_row)
        salario_uma40_array.append(round(salario_array[i]/uma_row, 1))

        #Find the last day of the current month
        lastDayOfMonth = time_series_df.iloc[i].replace(day=1) + relativedelta(day=31) # Even if the month does not have 31 days, the relative delta manages
        # Calculate the number of remaining days
        DiasCubiertosMensuales_array.append((lastDayOfMonth - time_series_df.iloc[i]).days + 1)

        if salario_uma40_array[i] >= 4.01:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -2]
        elif salario_uma40_array[i] >= 3.51:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -3]
        elif salario_uma40_array[i] >= 3:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -4]
        elif salario_uma40_array[i] >= 2.51:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -5]
        elif salario_uma40_array[i] >= 2:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -6]
        elif salario_uma40_array[i] >= 1.51:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -7]
        elif salario_uma40_array[i] >= 1:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -8]
        else:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -9]

        PORCENTAJE_PAGO_array.append(CUOTA_PATRONAL + CUOTA_OBRERA)

    PORCENTAJE_PAGO_array_string = map(lambda x: round(x*1e2, 2)/1e2, PORCENTAJE_PAGO_array)
    # Wtih map we apply our custom function to each element specified in the iloc
    # indexation, we remove the '%' with the replace function and divide by 1e2
    time_series_df = time_series_df.map(lambda x: x.strftime('%Y-%m-%d'))
    salario_asignado_mensual = salario_array * DiasCubiertosMensuales_array
    mes_de_pago = time_series_df.map(lambda x:pd.to_datetime(x).month)
    mes_de_pago.replace({1: 'Enero', 2: 'Febrero', 3:'Marzo', 4:'Abril', 5:'Mayo', 6:'Junio', 7:'Julio', 8: 'Agosto', 9: 'Septiembre', 10:'Octubre', 11:'Noviembre', 12:'Diciembre'}, inplace=True)

    # # Pago Mensual
    PAGO_MENSUAL_array = (salario_asignado_mensual * PORCENTAJE_PAGO_array).round(2)
    pago_acumulado = np.cumsum(PAGO_MENSUAL_array)
    tabla_pago40 = pd.DataFrame({'Número de Pago': np.arange(1, durata + 2) ,
                                 'Fechas de Pago': time_series_df,
                                 'Día de Inicio de Cobertura': time_series_df.map(lambda x:pd.to_datetime(x).day),
                                'Mes de Pago': mes_de_pago,
                                'Año de Pago': time_series_df.map(lambda x:pd.to_datetime(x).year),
                                'Salario Diario Asignado': salario_array,
                                'Relación Salario/UMA': salario_uma40_array,
                                'Días cubiertos': DiasCubiertosMensuales_array,
                                'Salario Asignado Mensual': salario_asignado_mensual,
                                '% de pago respecto del salario asignado': PORCENTAJE_PAGO_array_string,
                                'Pago Mensual': PAGO_MENSUAL_array,
                                'Pago Acumulado': pago_acumulado,
                                'Salario Mínimo del año en curso': salarios_minimos_array,
                                'Salario Máximo del año en curso': salarios_maximos_array})

    tabla_pago40.set_index(np.arange(1, durata + 2), inplace = True)
    tabla_pago40_string = tabla_pago40.copy()
    tabla_pago40_string[['Salario Diario Asignado', 'Salario Asignado Mensual', 'Pago Mensual', 'Pago Acumulado', 'Salario Mínimo del año en curso', 'Salario Máximo del año en curso']] = tabla_pago40[['Salario Diario Asignado', 'Salario Asignado Mensual', 'Pago Mensual', 'Pago Acumulado', 'Salario Mínimo del año en curso', 'Salario Máximo del año en curso']].map("${:,.2f}".format)
    tabla_pago40_string['% de pago respecto del salario asignado'] = (tabla_pago40['% de pago respecto del salario asignado']).map("{:.2%}".format)

    return tabla_pago40, tabla_pago40_string

# Simulación de Pagos en Modalidad 40, solo un mes de variable

In [18]:
salario = 2593.5 #@param {type:"number"}
start_date = "2023-02-03" #@param {type:"date"}
start_date = datetime.strptime(start_date, "%Y-%m-%d").date()

durata = 24 #@param {type:"integer"}

pagos_num, pagos_str = pagos40(salario, start_date, durata)
pagos_str

,Número de Pago,Fechas de Pago,Día de Inicio de Cobertura,Mes de Pago,Año de Pago,Salario Diario Asignado,Relación Salario/UMA,Días cubiertos,Salario Asignado Mensual,% de pago respecto del salario asignado,Pago Mensual,Pago Acumulado,Salario Mínimo del año en curso,Salario Máximo del año en curso
1,1,2023-02-03,3,Febrero,2023,"$2,593.50",25.0,26,"$67,431.00",11.17%,"$7,529.35","$7,529.35",$207.44,"$2,593.50"
2,2,2023-03-01,1,Marzo,2023,"$2,593.50",25.0,31,"$80,398.50",11.17%,"$8,977.30","$16,506.65",$207.44,"$2,593.50"
3,3,2023-04-01,1,Abril,2023,"$2,593.50",25.0,30,"$77,805.00",11.17%,"$8,687.71","$25,194.36",$207.44,"$2,593.50"
4,4,2023-05-01,1,Mayo,2023,"$2,593.50",25.0,31,"$80,398.50",11.17%,"$8,977.30","$34,171.66",$207.44,"$2,593.50"
5,5,2023-06-01,1,Junio,2023,"$2,593.50",25.0,30,"$77,805.00",11.17%,"$8,687.71","$42,859.37",$207.44,"$2,593.50"
6,6,2023-07-01,1,Julio,2023,"$2,593.50",25.0,31,"$80,398.50",11.17%,"$8,977.30","$51,836.67",$207.44,"$2,593.50"
7,7,2023-08-01,1,Agosto,2023,"$2,593.50",25.0,31,"$80,398.50",11.17%,"$8,977.30","$60,813.97",$207.44,"$2,593.50"
8,8,2023-09-01,1,Septiembre,2023,"$2,593.50",25.0,30,"$77,805.00",11.17%,"$8,687.71","$69,501.68",$207.44,"$2,593.50"
9,9,2023-10-01,1,Octubre,2023,"$2,593.50",25.0,31,"$80,398.50",11.17%,"$8,977.30","$78,478.98",$207.44,"$2,593.50"
10,10,2023-11-01,1,Noviembre,2023,"$2,593.50",25.0,30,"$77,805.00",11.17%,"$8,687.71","$87,166.69",$207.44,"$2,593.50"


## Crecimiento anual:


1. Año de inicio
2. Valor de pensión
3. Año Final




In [19]:
def crecimiento_anual_pension(ano_pension, pension_final, year_max):
    tabla = 'TABLAS/TABLA_INFLACION.txt'
    tabla = pd.read_csv(tabla, sep = '\t')
    year_max = min(2040, year_max)
    num_years = year_max - ano_pension
    year_array = np.arange(0, num_years + 1) + ano_pension
    idx_inicial = tabla.index[tabla['AÑO'] == ano_pension].tolist()[0]
    idx_final = tabla.index[tabla['AÑO'] == ano_pension + num_years].tolist()[0]

    inflacion_anual_numerico = tabla.iloc[idx_inicial:idx_final, 1]
    inflacion_anual_numerico = inflacion_anual_numerico.map(lambda x: float(x.replace('%','')))
    inflacion_anual_numerico = pd.concat([pd.Series([0]), inflacion_anual_numerico])

    inflacion_anual_acumulada = np.cumsum(inflacion_anual_numerico)
    pension_mensual = (inflacion_anual_acumulada/1e2 + 1) * pension_final

    df = pd.DataFrame({
        'Año': year_array.astype(int),
        'Inflación Anual': inflacion_anual_numerico,
        'Inflación Anual Acumulada' : inflacion_anual_acumulada,
        'Pensión recibida mensual' : pension_mensual
    })

    return df.round(2)

In [20]:
ano_inicio = 2025 #@param {type:"integer"}
valor_pension = 56110 #@param {type:"integer"}
year_max = 2040 #@param {type:"integer"}

In [21]:
df = crecimiento_anual_pension(ano_inicio, valor_pension, year_max)

def balance_futuro(fecha_inicio_total, ano_final, dict_balance, pagos = None):
    valor_pension1, inicio_pension1, crecimiento_anual, aguinaldos, fecha_nacimiento_date = dict_balance.values()
    titulo_1 = f"Suma Recibida Inicial: ${valor_pension1}\nInicio de Pensión: {inicio_pension1}" # + '. Inicio: ' + str(inicio_pension1)
    fecha_inicio_total = fecha_inicio_total.replace(day = 1)
    fecha_final = date(ano_final, 12, 1)
    second_date = fecha_inicio_total + relativedelta(months=1, day=1) # adds one month and sets the day to 1.

    rd = relativedelta(fecha_final, fecha_inicio_total)
    months_rd = rd.years * 12 + rd.months + 1 # We add +1 to include the final month of december

    risultato = pd.date_range(start=fecha_inicio_total, periods=months_rd, freq='MS')
    # Convert DatetimeIndex to list
    risultato = [d.date() for d in risultato]

    pension1_list = list()
    aguinaldo_array = list()
    perc_aguinaldo = list()
    pension_array_completo = list()

    aguinaldo_perc = aguinaldo_num.iloc[:, -1]

    if len(np.unique(aguinaldo_perc)) > 1:
        aguinaldo_ref = aguinaldo_perc[0]
        nuevo_perc_pension = (1 + (aguinaldo_perc - aguinaldo_ref) * 0.869584149) #* crecimiento_anual.iloc[:, -1]
        nuevo_perc_pension.reset_index(drop=True, inplace=True)
        crecimiento_anual.reset_index(drop=True, inplace=True)
        crecimiento_anual.iloc[:, -1] *= nuevo_perc_pension
        aguinaldo_num.iloc[:, -2] *= nuevo_perc_pension

    for idx, val in enumerate(risultato):
        if val < inicio_pension1.replace(day=1):
            pension1_list.append(0)
            aguinaldo_array.append(0)
            perc_aguinaldo.append(0)
            pension_array_completo.append(0)
        elif val == inicio_pension1.replace(day=1):
            pension1_list.append((1 - (inicio_pension1.day - 1) / 365 * 12) * valor_pension1)
            index_year = crecimiento_anual['Año'].values.tolist().index(val.year)
            if val.month == 11:
                aguinaldo_array.append(aguinaldos.iloc[index_year, 1])
                perc_aguinaldo.append(aguinaldos.iloc[index_year, 2])
            else:
                aguinaldo_array.append(0)
                perc_aguinaldo.append(0)
            pension_array_completo.append(np.sum(pension1_list) + np.sum(aguinaldo_array))
        else:
            index_year = crecimiento_anual['Año'].values.tolist().index(val.year)
            if val.month == 11:
                aguinaldo_array.append(aguinaldos.iloc[index_year, 1])
                perc_aguinaldo.append(aguinaldos.iloc[index_year, 2])
            else:
                aguinaldo_array.append(0)
                perc_aguinaldo.append(0)
            pension1_list.append(crecimiento_anual.iloc[index_year, -1])
            pension_array_completo.append(np.sum(pension1_list) + np.sum(aguinaldo_array))

    suma = list(map(sum, zip(aguinaldo_array, pension1_list)))

    tabla_balance_num = pd.DataFrame({'Fecha': risultato,
                                    'Edad' : pd.Series(risultato).map(lambda x: round((x - fecha_nacimiento_date).days/365.25, 2)),
                                  'Pensión Recibida Mensual' : pension1_list,
                                  'Aguinaldo' : aguinaldo_array,
                                  '% de Aguinaldo' : perc_aguinaldo,
                                  'Pensión Recibida Total': suma,
                                  titulo_1: pension_array_completo})

    tabla_balance_num = tabla_balance_num.round(2)

    def special_formatting_column(col, custom_format):
        out = pd.Series(col).map(lambda x: custom_format.format(x))
        return out

    tabla_balance_str = pd.DataFrame({'Fecha': pd.Series(risultato).apply(lambda x: x.strftime('%m-%Y')),
                                    'Edad' : pd.Series(risultato).map(lambda x: round((x - fecha_nacimiento_date).days/365.25, 2)),
                                  'Pensión Recibida Mensual' : special_formatting_column(pension1_list, "${:,.2f}"),
                                  'Aguinaldo' : special_formatting_column(aguinaldo_array, "${:,.2f}"),
                                  '% de Aguinaldo' : pd.Series(perc_aguinaldo).apply(lambda x: "{:.0f}%".format(x * 1e2)),
                                  'Pensión Recibida Total' : special_formatting_column(suma, "${:,.2f}"),
                                    titulo_1 : special_formatting_column(pension_array_completo, "${:,.2f}")})

    if pagos is not None:
        pagos_array = np.full(len(risultato), 0)
        pagos_num = pagos
        pagos_balance = pagos_num.loc[:, ['Fechas de Pago', 'Pago Mensual', 'Pago Acumulado']]
        inicio_pago = pagos_balance.loc[1, 'Fechas de Pago']
        inicio_pago = pd.to_datetime(inicio_pago)
        inicio_pago = inicio_pago.to_period("M")
        idx_final = len(pagos_balance)

        idx = pd.to_datetime(risultato).to_period("M")

        idx.to_list().index(inicio_pago)

        tabla_balance_num['Pago Acumulado en Mod. 40'] = pagos_array[idx.to_list().index(inicio_pago) : len(pagos_num) + idx.to_list().index(inicio_pago)] = pagos_balance['Pago Acumulado'].reset_index(drop=True)
        tabla_balance_num['Pago Acumulado en Mod. 40'] = tabla_balance_num['Pago Acumulado en Mod. 40'].fillna(0)
        tabla_balance_str['Pago Acumulado en Mod. 40'] = tabla_balance_num['Pago Acumulado en Mod. 40']
        tabla_balance_str['Pago Acumulado en Mod. 40'] = tabla_balance_str['Pago Acumulado en Mod. 40'].map("${:,.2f}".format)

        tabla_balance_num['Pago Mensual Mod. 40'] = pagos_array[idx.to_list().index(inicio_pago) : len(pagos_num) + idx.to_list().index(inicio_pago)] = pagos_balance['Pago Mensual'].reset_index(drop=True)
        tabla_balance_num['Pago Mensual Mod. 40'] = tabla_balance_num['Pago Mensual Mod. 40'].fillna(0)
        tabla_balance_str['Pago Mensual Mod. 40'] = tabla_balance_num['Pago Mensual Mod. 40']
        tabla_balance_str['Pago Mensual Mod. 40'] = tabla_balance_str['Pago Mensual Mod. 40'].map("${:,.2f}".format)

        pension_array_completo = tabla_balance_num["Pensión Recibida Total"] - tabla_balance_num["Pago Mensual Mod. 40"]
        tabla_balance_num[titulo_1] = pension_array_completo.cumsum().round(2)
        tabla_balance_str[titulo_1] = tabla_balance_num[titulo_1].map("${:,.2f}".format)

        columns_to_move = ['Pago Mensual Mod. 40', 'Pago Acumulado en Mod. 40']
        position = 2
        # Get list of current columns
        cols = list(tabla_balance_num.columns)

        for col in columns_to_move:
            cols.remove(col)

            # Insert the columns at the specified position
        for i, col in enumerate(columns_to_move):
            cols.insert(position + i, col)

        tabla_balance_num = tabla_balance_num[cols]
        tabla_balance_str = tabla_balance_str[cols]

    return tabla_balance_num, tabla_balance_str

# Agregando Semanas con Mod. 40

In [26]:
# BALANCE
# Mod. 40
salario_asignado = 2828.5 #@param {type:"number"}
start_date = "2025-03-03" #@param {type:"date"}
start_date = datetime.strptime(str(start_date), "%Y-%m-%d").date()
durata = 23 #@param {type:"integer"}
pension_recibida = 56110 #@param {type:"number"}

pagos_num, pagos_str = pagos40(salario_asignado, start_date, durata)

inicio_pago_pension = "2025-02-07" #@param {type:"date"}
inicio_pago_pension = datetime.strptime(str(inicio_pago_pension), "%Y-%m-%d").date()
ano_final_pension = 2030 #@param {type:"integer"}

aguinaldo_num, aguinaldo_str = tabla_aguinaldo(fecha_inicio = inicio_pago_pension,
                                               pension_final = pension_recibida,
                                               year_max = ano_final_pension,
                                               perc_asignaciones_familiares = 0.15,
                                               year_cambio_asignaciones = 2035,
                                               perc_restar = 0.10)

dict1 = {'$ Pensión Recibida': pension_recibida,
         'Fecha de Inicio': inicio_pago_pension,
         'Crecimiento Anual de Pensión' : crecimiento_anual_pension(inicio_pago_pension.year, pension_recibida, ano_final_pension),
         'Aguinaldo': aguinaldo_num,
         'Fecha de Nacimiento': Usuario.fecha_nacimiento}

tabla_balance_num, tabla_balance_str = balance_futuro(fecha_inicio_total = start_date,
                                                      ano_final = ano_final_pension,
                                                      dict_balance = dict1,
                                                      pagos = pagos_num)
type()display(tabla_balance_str)

,Fecha,Edad,Pago Mensual Mod. 40,Pago Acumulado en Mod. 40,Pensión Recibida Mensual,Aguinaldo,% de Aguinaldo,Pensión Recibida Total,Suma Recibida Inicial: $56110\nInicio de Pensión: 2025-02-07
0,03-2025,60.38,"$10,948.08","$10,948.08","$56,110.00",$0.00,0%,"$56,110.00","$45,161.92"
1,04-2025,60.47,"$11,325.60","$22,273.68","$56,110.00",$0.00,0%,"$56,110.00","$89,946.32"
2,05-2025,60.55,"$11,703.12","$33,976.80","$56,110.00",$0.00,0%,"$56,110.00","$134,353.20"
3,06-2025,60.63,"$11,325.60","$45,302.40","$56,110.00",$0.00,0%,"$56,110.00","$179,137.60"
4,07-2025,60.71,"$11,703.12","$57,005.52","$56,110.00",$0.00,0%,"$56,110.00","$223,544.48"
5,08-2025,60.80,"$11,703.12","$68,708.64","$56,110.00",$0.00,0%,"$56,110.00","$267,951.36"
6,09-2025,60.88,"$11,325.60","$80,034.24","$56,110.00",$0.00,0%,"$56,110.00","$312,735.76"
7,10-2025,60.97,"$11,703.12","$91,737.36","$56,110.00",$0.00,0%,"$56,110.00","$357,142.64"
8,11-2025,61.05,"$11,325.60","$103,062.96","$56,110.00","$42,728.15",15%,"$98,838.15","$444,655.19"
9,12-2025,61.13,"$11,703.12","$114,766.08","$56,110.00",$0.00,0%,"$56,110.00","$489,062.07"


In [33]:
tabla_salarios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, 0 to 29
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Fecha Inicial  29 non-null     object
 1   Fecha Final    29 non-null     object
 2   Sueldo         29 non-null     object
dtypes: object(3)
memory usage: 2.0+ KB


In [25]:
def nuevo_salario_promedio(tabla_salarios, salario, start_date, durata, semanas_totales, TABLA_SALARIO_PROMEDIO):
    historia_laboral_normal = tabla_salarios.copy()
    pagos = pagos40(salario, start_date, durata)

    position = 0
    for idx in range(len(historia_laboral_normal)):
        if start_date < historia_laboral_normal.iloc[idx, 1]:
            position = idx
        else:
            break
    if start_date >= TABLA_SALARIO_PROMEDIO.iloc[0, 1]:
        semanas_restar = 0
    else:
        semanas_restar = TABLA_SALARIO_PROMEDIO.iloc[:position+1, 4].sum() - (historia_laboral_normal.iloc[position + 1, 1] - start_date).days / 7

    semanas_reconocidas_nuevas = round(semanas_totales - semanas_restar, 1)

    historia_laboral_recortada = historia_laboral_normal.iloc[position:, :].copy()

    if start_date == historia_laboral_recortada.iloc[0, 0]:
        historia_laboral_recortada.drop(historia_laboral_recortada.index[0], inplace=True)
    else:
        historia_laboral_recortada.iloc[0, 1] = min(start_date, historia_laboral_recortada.iloc[0, 1])

    dias_cobertura = pagos[0]['Días cubiertos'].sum()
    semanas_cobertura_mod40 = round(dias_cobertura / 7, 2)
    semanas_final = round(semanas_cobertura_mod40 + semanas_reconocidas_nuevas)

    if dias_cobertura >= 1750:
        salario_promedio_final = salario

    else:
        semanas_contar = 250 - semanas_cobertura_mod40
        salario_normal_promedio, tabla_salario_normal_promedio = salario_promedio_fcn(semanas_contar, semanas_reconocidas_nuevas, historia_laboral_recortada)
        salario_acumulado_normal = salario_normal_promedio * semanas_contar * 7
        salario_acumulado_mod40 = salario * dias_cobertura
        salario_promedio_final = round((salario_acumulado_mod40 + salario_acumulado_normal) / 1750, 2)

    return semanas_final, salario_promedio_final

# durata = 56
risultati = nuevo_salario_promedio(tabla_salarios, salario, start_date, durata, Usuario.semanas_totales, TABLA_SALARIO_PROMEDIO)
print(f"Semanas al terminar: {risultati[0]}")
print(f"Salario promedio final: {risultati[1]}")
display(tabla_balance_str)

TypeError: '<' not supported between instances of 'datetime.date' and 'str'

# Tabla Pivot de Pagos Mod. 40

In [ ]:
def pivot_pagos40(df):
    pivot_table = pd.pivot_table(data=df,
                index = 'Mes de Pago',
                values = 'Pago Mensual',
                columns = 'Año de Pago',
                aggfunc = 'sum',
                fill_value=0,
                margins=True,
                margins_name='Total')
    # Rename margin row and column
    pivot_table.rename(index={pivot_table.index[-1]: 'Total por mes'},
                    columns={pivot_table.columns[-1]: 'Total por año'},
                    inplace=True)

    # Define the desired order
    order = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre', 'Total por mes']

    # Reindex the pivot table
    ordered_pivot_table = pivot_table.reindex(index=order)

    # Convert float values to currency strings
    ordered_pivot_table_string = ordered_pivot_table.map("${:,.2f}".format)

    return ordered_pivot_table, ordered_pivot_table_string

risultati = pivot_pagos40(pagos_num)
pivot_table = risultati[0]
pivot_table_string = risultati[1]
pivot_table_string

Año de Pago,2025,2026,2027,Total por año
Mes de Pago,,,,
Enero,$0.00,"$12,659.74","$13,615.49","$26,275.23"
Febrero,$0.00,"$11,434.61","$12,297.87","$23,732.48"
Marzo,"$10,948.08","$12,659.74",$0.00,"$23,607.82"
Abril,"$11,325.60","$12,251.36",$0.00,"$23,576.96"
Mayo,"$11,703.12","$12,659.74",$0.00,"$24,362.86"
Junio,"$11,325.60","$12,251.36",$0.00,"$23,576.96"
Julio,"$11,703.12","$12,659.74",$0.00,"$24,362.86"
Agosto,"$11,703.12","$12,659.74",$0.00,"$24,362.86"
Septiembre,"$11,325.60","$12,251.36",$0.00,"$23,576.96"


## Cálculo de edad que obtiene al entrar en mod. 40

In [32]:
# prompt: create a function that calculates 3 parameters: start_date, num_months, and date_birth, the return should be a tuple, with the first element calculating start_date + num_month and the second element calculate the age by substracting the first element from date_birth
def calculate_date_and_age(start_date, num_months, date_birth):
    future_date = start_date + relativedelta(months = 1)
    future_date = future_date.replace(day = 1)
    future_date = (future_date + relativedelta(months = num_months)).date()
    future_date = future_date - relativedelta(days=1)
    if isinstance(date_birth, date):
        age_unrounded = round((future_date - date_birth).days / 365.25, 2)
    else:
        age_unrounded = round((future_date - date_birth.date()).days / 365.25, 2)
    age_rounded = round(age_unrounded)
    return future_date, age_unrounded, age_rounded

start_date = datetime(2024, 10, 17)
num_months = 1
date_birth = fecha_nacimiento_date #datetime(1962, 4, 23)

# Call the function with the given parameters
risultati, eta_non, eta = calculate_date_and_age(start_date, num_months, date_birth)
display(risultati)
display(eta_non)
display(eta)

NameError: name 'fecha_nacimiento_date' is not defined

# Tabla Pivot con Plotly

In [ ]:
import plotly.graph_objects as go

def heatmap_pagos40(pivot_table):
    # df should have 'Mes de Pago' as index, 'Año de Pago' as columns, and 'Pago Mensual' as values
    # Melt the DataFrame to long format
    df = pivot_table.copy()
    df.drop('Total por mes', axis=0, inplace=True)
    df.drop('Total por año', axis=1, inplace=True)
    df_melted = df.reset_index().melt(id_vars=['Mes de Pago'], var_name='Año de Pago', value_name='Pago Mensual')
    # Create the heatmap
    fig = go.Figure(data=go.Heatmap(
        x=df_melted['Año de Pago'],
        y=df_melted['Mes de Pago'],
        z=df_melted['Pago Mensual'],
        colorscale='Turbo',
        colorbar=dict(
            tickformat="$,.0f"
        ),
        text=df_melted.apply(lambda row: f'Mes: {row["Mes de Pago"]}<br>Año: {row["Año de Pago"]}<br>Pago Mensual: ${row["Pago Mensual"]:.2f}', axis=1),

        hoverinfo='text',
    ))

    fig.update_layout(
        title='Pagos Mensuales',
        xaxis=dict(
            nticks=36,
            title='Año de Pago',
            tickmode='linear',
            tick0=0,
            dtick=1,
            tickformat='d'
        ),
        xaxis_nticks=36,
        yaxis_nticks=12,
        xaxis_title='Año de Pago',
        yaxis_title='Mes de Pago',
    )

    # Add text annotations
    for i in range(len(df_melted)):
        fig.add_annotation(
            x=df_melted['Año de Pago'][i],
            y=df_melted['Mes de Pago'][i],
            text=f"${df_melted['Pago Mensual'][i]:,.0f}",
            showarrow=False,
            font=dict(size=16, color = 'white')
        )

    return fig.show()

heatmap_pagos40(pivot_table)

# Resumen de costos por meses en mod. 40

In [46]:
5# Mod. 40
salario_asignado = 2828.5 #@param {type:"number"}
start_date = "2025-03-03" #@param {type:"date"}
start_date = datetime.strptime(str(start_date), "%Y-%m-%d").date()
durata = 58 #@param {type:"integer"}
perc_asignaciones = 0.15 #@param {type:"number"}


In [47]:
pagos_num, pagos_str = pagos40(salario_asignado, start_date, durata)
pagos_num

def costos_40(fecha_nacimiento_date, salario_asignado, meses_inferior, meses_superior, start_date, perc_asignaciones, nuevo_salario_promedio_dict = None):
    duracion = (1 + meses_superior - meses_inferior)
    meses_array = np.arange(meses_inferior, meses_superior + 1)
    inicio_array = np.full(duracion, start_date)
    final_array = list()
    solicitud_array = list()
    pagos = list()
    edad_array = list()
    pension_final_array = list()
    age_60 = fecha_nacimiento_date + relativedelta(years=60, days=1)

    for i, idx in enumerate(meses_array):  # Use enumerate to get the index i
        _, pagos_str = pagos40(salario_asignado, start_date, idx)
        pagos.append(pagos_str['Pago Acumulado'].iloc[-1])
        final_array.append(start_date.replace(day=1) + relativedelta(months=meses_array[i] + 1) - timedelta(days=1))  # Use i instead of idx
        solicitud_array.append(max(final_array[i] + timedelta(days=1), age_60))  # Use i instead of idx
        edad_array.append(round((solicitud_array[i] - fecha_nacimiento_date).days / 365.25, 2))  # Use i instead of idx

    edad_redondeada = np.round(np.array(edad_array), 0).astype(int)
    perc_pension_vejez_array = pd.Series(edad_redondeada).apply(lambda x: porcentaje_vejez_fcn(x) * 1e2)
    df_costos = pd.DataFrame({
        'Inicio de pagos' : inicio_array,
        'Meses pagando' : meses_array + 1,
        'Final de pagos': final_array,
        'Solicitud de pensión': solicitud_array,
        'Edad al solicitar pensión' : edad_array,
        'Edad redondeada' : edad_redondeada,
        '% Pensión de acuerdo a la edad': perc_pension_vejez_array,
        'Pago Total': pagos
        })

    if nuevo_salario_promedio_dict is not None:
        semanas_finales_array, salario250_array = list(), list()
        tabla_salarios, semanas_totales, TABLA_SALARIO_PROMEDIO = nuevo_salario_promedio_dict.values()
        for i, _ in enumerate(meses_array):
            semanas_finales, salario250 = nuevo_salario_promedio(tabla_salarios, salario_asignado, start_date, meses_array[i], semanas_totales, TABLA_SALARIO_PROMEDIO)
            semanas_finales_array.append(semanas_finales)
            salario250_array.append(salario250)
            pension_final_array.append(calculo_pension(edad_pension = edad_redondeada[i], year = solicitud_array[i].year, semanas_reconocidas = semanas_finales, salario_promedio = salario250, porcentaje_asignaciones = perc_asignaciones))

        df_salarios = pd.DataFrame({
            'Semanas totales': semanas_finales_array,
            'Salario Promedio': salario250_array,
            'Pensión Alcanzada': pd.Series(pension_final_array).apply(lambda x: "${:,.2f}".format(x))})

        df_total = pd.concat([df_costos, df_salarios], axis=1)
        df_total['Salario Promedio'] = df_total['Salario Promedio'].apply(lambda x: "${:,.2f}".format(x))

        df_total['Aumento respecto al mes anterior'] = (df_total['Pensión Alcanzada'].apply(lambda x: float(x.replace("$", "").replace(",", "")))).diff()
        df_total['Aumento respecto al mes anterior'] = df_total['Aumento respecto al mes anterior'].fillna(0)
        df_total['Aumento respecto al mes anterior'] = df_total['Aumento respecto al mes anterior'].apply(lambda x: "${:,.2f}".format(x))
        return df_total
    return df_costos

nuevo_salario_promedio_dict = {'Tabla de salarios': tabla_salarios,
                               'Semanas cotizadas finales': semanas_totales,
                               'Tabla de salario promedio': TABLA_SALARIO_PROMEDIO
                               }

df_corridapagos40 = costos_40(fecha_nacimiento_date = fecha_nacimiento_date,
                             salario_asignado = salario_asignado,
                             meses_inferior = 0,
                             meses_superior = durata,
                             start_date = start_date,
                             perc_asignaciones = perc_asignaciones,
                             nuevo_salario_promedio_dict = nuevo_salario_promedio_dict)
display(df_corridapagos40)

NameError: name 'semanas_totales' is not defined

In [ ]:
max_aumento = max(df_corridapagos40['Aumento respecto al mes anterior'])
posicion_max_aumento = df_corridapagos40[df_corridapagos40['Aumento respecto al mes anterior'] == max_aumento].index[0]

# Tabla Pivot Meses pagando y variacion de salario, como en Excel

In [ ]:
12 * [10]

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [ ]:
# Mod. 40
pagos_num, pagos_str = pagos40(salario, start_date, durata)
pagos_num

def costos_40_simplificado(df_corridapagos40, salario_asignado):
    risultato = df_corridapagos40[['Meses pagando', 'Pago Total', 'Pensión Alcanzada', 'Aumento respecto al mes anterior']]
    risultato.rename(columns={'Pensión Alcanzada': f'Salario Diario Asignado: ${salario_asignado} (Pensión Mensual Alcanzada)',
                              'Pago Total': f'Salario Diario Asignado: ${salario_asignado} (Inversión)'},
                     inplace=True)
    return risultato

df_corridapagos40_simplificado = costos_40_simplificado(df_corridapagos40, salario_asignado)
display(df_corridapagos40_simplificado.head())

from plotly.figure_factory import create_table
import plotly.offline as py
table = create_table(df_corridapagos40_simplificado)
py.iplot(table)

<ipython-input-35-20de6309f37c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  risultato.rename(columns={'Pensión Alcanzada': f'Salario Diario Asignado: ${salario_asignado} (Pensión Mensual Alcanzada)',


,Meses pagando,Salario Diario Asignado: $2828.5 (Inversión),Salario Diario Asignado: $2828.5 (Pensión Mensual Alcanzada),Aumento respecto al mes anterior
0,1,"$10,948.08","$20,054.00",$0.00
1,2,"$22,273.68","$21,064.40","$1,010.40"
2,3,"$33,976.80","$22,592.80","$1,528.40"
3,4,"$45,302.40","$23,930.50","$1,337.70"
4,5,"$57,005.52","$25,681.00","$1,750.50"
5,6,"$68,708.64","$27,076.10","$1,395.10"
6,7,"$80,034.24","$28,418.30","$1,342.20"
7,8,"$91,737.36","$29,821.30","$1,403.00"
8,9,"$103,062.96","$31,165.10","$1,343.80"
9,10,"$114,766.08","$32,527.60","$1,362.50"


In [ ]:
meses = [0, 5]
salarios = [1000, 2000, 3000]

nuevo_salario_promedio_dict = {'Tabla de salarios': tabla_salarios,
                               'Semanas cotizadas finales': semanas_totales,
                               'Tabla de salario promedio': TABLA_SALARIO_PROMEDIO
                               }
costos_40(fecha_nacimiento_date, salario_asignado, 0, 20, start_date, perc_asignaciones, nuevo_salario_promedio_dict)

def pivot_pagos_costos(meses, salarios, fecha_nacimiento_date, start_date, perc_asignaciones, nuevo_salario_promedio_dict):
    lista_pagos = list()
    lista_pensiones = list()
    meses_pagados = pd.Series(range(meses[0], meses[1] + 1))
    for salarios_idx in salarios:
        df_corridapagos40 = costos_40(fecha_nacimiento_date, salarios_idx, meses[0], meses[1], start_date, perc_asignaciones, nuevo_salario_promedio_dict)
        resultados = costos_40_simplificado(df_corridapagos40, salarios_idx)
        pagos = resultados.iloc[:, 1]
        pensiones = resultados.iloc[:, -2]
        lista_pagos.append(pagos)
        lista_pensiones.append(pensiones)

    pagos = pd.concat(lista_pagos, axis=1)
    pagos['Meses en Mod. 40'] = meses_pagados
    pagos = pagos.set_index('Meses en Mod. 40')
    pensiones = pd.concat(lista_pensiones, axis=1)
    pensiones['Meses en Mod. 40'] = meses_pagados
    pensiones = pensiones.set_index('Meses en Mod. 40')
    return pagos, pensiones

meses = [2, 9]
salarios = [1000, 2000, 1250, 1450]
pagos, pensiones = pivot_pagos_costos(meses, salarios, fecha_nacimiento_date, start_date, perc_asignaciones, nuevo_salario_promedio_dict)
display(pagos)
display(pensiones)

,Salario Diario Asignado: $1000 (Inversión),Salario Diario Asignado: $2000 (Inversión),Salario Diario Asignado: $1250 (Inversión),Salario Diario Asignado: $1450 (Inversión)
Meses en Mod. 40,,,,
2,"$12,012.30","$24,024.60","$15,015.37","$17,417.83"
3,"$16,016.40","$32,032.80","$20,020.49","$23,223.77"
4,"$20,153.97","$40,307.94","$25,192.45","$29,223.25"
5,"$24,291.54","$48,583.08","$30,364.41","$35,222.73"
6,"$28,295.64","$56,591.28","$35,369.53","$41,028.67"
7,"$32,433.21","$64,866.42","$40,541.49","$47,028.15"
8,"$36,437.31","$72,874.62","$45,546.61","$52,834.09"
9,"$40,574.88","$81,149.76","$50,718.57","$58,833.57"


,Salario Diario Asignado: $1000 (Pensión Mensual Alcanzada),Salario Diario Asignado: $2000 (Pensión Mensual Alcanzada),Salario Diario Asignado: $1250 (Pensión Mensual Alcanzada),Salario Diario Asignado: $1450 (Pensión Mensual Alcanzada)
Meses en Mod. 40,,,,
2,"$20,013.00","$21,381.50","$20,435.90","$20,612.10"
3,"$20,350.40","$22,103.70","$20,751.10","$21,198.80"
4,"$20,830.40","$23,348.00","$21,544.50","$21,960.30"
5,"$21,173.40","$24,264.30","$21,879.10","$22,397.70"
6,"$21,497.10","$25,143.20","$22,341.70","$22,968.80"
7,"$21,694.40","$26,067.10","$22,668.20","$23,574.50"
8,"$22,017.50","$26,947.20","$23,129.00","$24,147.20"
9,"$22,325.20","$27,830.90","$23,579.00","$24,713.10"


In [ ]:
def posicion_max_aumento(pensiones):
    # Iterate through each column
    for col in pensiones.columns:
        # Apply the transformations element-wise to strings within the column
        pensiones[col] = pensiones[col].astype(str).str.replace('$', '').str.replace(',', '').astype(float)

    # Calculate the difference
    pensiones = pensiones.diff()
    pensiones = pensiones.fillna(0)

    max_aumento = np.max(pensiones, axis = 0)
    posicion = pensiones.idxmax()

    return np.array(posicion)

posicion = posicion_max_aumento(pensiones)

display(posicion[-1])

4

In [ ]:
import plotly.graph_objects as go
import numpy as np
def create_3dplot(meses, salarios, dict_modalidad):
    parameter, texto, default_color, start_date = dict_modalidad.values()
    # Create data
    x = np.arange(meses[0], meses[1] + 1)
    y = salarios
    # Highlight the third point (index 2)
    highlight_index = 2
    X, Y = np.meshgrid(x, y)

    # highlight_color = 'pink'

    Z = parameter.map(lambda x: float(x.replace('$', '').replace(',', '')))
    # colors = [default_color if i != highlight_index else highlight_color for i in range(len(Z))]
    Z = Z.values.flatten()

    # Create figure
    fig = go.Figure(data=[go.Scatter3d(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z,
        mode='markers',
        line=dict(color=default_color, width=1),
        text= [f"Fecha de Inicio: {start_date.strftime('%m-%Y')}<br>Meses en Mod. 40: {x_val}<br>Salario Asignado: ${y_val} <br>{texto}: ${z_val}"
            for x_val, y_val, z_val in zip(X.flatten(), Y.flatten(), Z.flatten())],  # Set Z values as text
        hoverinfo='text',
    )])

    # Customize layout
    fig.update_layout(
        title=f'{texto} de acuerdo al num. de meses cotizando y sueldo asignado',
        scene=dict(
            xaxis_title='Meses en Mod. 40',
            yaxis_title='Salario Asignado',
            zaxis_title=f'{texto}',
            xaxis=dict(nticks=len(x)),
            yaxis=dict(
                tickvals=y,
                ticktext=[f"${salario:,.0f}" for salario in y]
                )
        ),
        width=700,
        height=550,
        title_y = 0.95,
        margin=dict(
        l=0,  # left margin
        r=0,  # right margin
        t=0,  # top margin (adjust this value to reduce the space between title and chart)
        b=0   # bottom margin
    ),
        paper_bgcolor='rgba(206,234,237,255)'
    )
    # Display plot
    return fig.show()

meses = [0, 3]
salarios = [2015]
pagos, pensiones = pivot_pagos_costos(meses, salarios, fecha_nacimiento_date, start_date, perc_asignaciones, nuevo_salario_promedio_dict)

# posicion_max_aumento(pensiones)
dict_modalidad_pagos = {
    'parametro': pagos.T,
    'texto': 'Inversión Total',
    'color': 'red',
    'fecha de inicio': start_date
}

dict_modalidad_pension = {
    'parametro': pensiones.T,
    'texto': 'Pensión Mensual',
    'color': 'lightgreen',
    'fecha de inicio': start_date
}
create_3dplot(meses, salarios, dict_modalidad_pension)

In [ ]:
a = np.array([1, 6])
a.fill(2)
a

array([2, 2])

In [ ]:
from datetime import datetime
from fpdf import FPDF
from fpdf.fonts import FontFace

class PDF(FPDF):
    def header(self):
        # Setting font: helvetica bold 15
        self.set_font("helvetica", style="B", size=5)
        self.set_text_color(0, 0, 255)  # Blue
        # Moving cursor to the right:
        self.cell(5)
        # Printing title:
        self.cell(0, 10,
                  "Portal para obtención de semanas cotizadas",
                  align="L",
                  link="https://serviciosdigitales.imss.gob.mx/semanascotizadas-web/usuarios/IngresoAsegurado")
        self.set_text_color(0, 0, 0)  # Black
        self.cell(0, 10, 'Fecha de generación de reporte: ' + str(datetime.today().date()),
                 new_x="LMARGIN", new_y="NEXT", align='R')
        # Performing a line break:
        self.ln(5)

    def footer(self):
        # Position cursor at 1.5 cm from bottom:
        self.set_y(-15)
        # Setting font: helvetica italic 8
        self.set_font("helvetica", style="I", size=8)
        self.set_text_color(128)
        # Printing page number:
        self.cell(0, 10, f"Página {self.page_no()} de {{nb}}", align="C")

def test_pdfv2(*args):
    if not args:
        raise ValueError("At least one table is required.")

    pdf = PDF(orientation="L")
    pdf.set_auto_page_break(auto=True, margin=15)

    for i, table_data in enumerate(args):
        # Extract title and dataframe from the current table
        title, leyenda, df, col_widths, title_size = table_data.values()

        # Add a new page for each table
        pdf.add_page()
        # Add title
        pdf.set_font("Helvetica", size = title_size)
        if i == 0:
            pdf.cell(0, 10, nombre, align="C")
            pdf.ln(10)  # Add space after title

        pdf.cell(0, 10, title, align="C")  # Center-aligned title
        if leyenda:
            pdf.set_font("Helvetica", style="I", size = 5)
            pdf.ln(10)
            pdf.free_text_annotation(text = leyenda)

        # Add table
        pdf.set_font("Helvetica", size=9)
        pdf.ln(10)  # Add some space before the table
        black = (0, 0, 0)
        grey = (198, 204, 199)
        pdf.set_text_color(*black)
        headings_style = FontFace(emphasis="BOLD", color=black, fill_color=grey)

        # Set column widths manually for better control:
        col_widths = col_widths  # Adjust as needed
        with pdf.table(col_widths=col_widths, text_align="CENTER", headings_style=headings_style) as table:
            # Add header row:
            row = table.row()
            for column in df.columns:
                row.cell(column)
            # Add data rows:
            for _, data_row in df.iterrows():
                row = table.row()
                for value in data_row:
                    row.cell(str(value))  # Ensure all values are strings

    # Generate and return PDF bytes
    pdf_bytes = pdf.output()
    return pdf_bytes

primera_tabla = {'Titulo' : 'Tabla de Salarios Promedios',
                 'Leyenda': None,
                 'Tabla' : FechasGenerales,
                 'Ancho de Columnas' : [30, 30, 25, 25, 40, 30],
                 'Tamaño de letra del título': 18}

segunda_tabla = {'Titulo' : 'Historia Laboral Desglosada',
                 'Leyenda': None,
                 'Tabla' : HistoriaLaboralTable,
                 'Ancho de Columnas' : [30, 30, 25, 25, 40, 30],
                 'Tamaño de letra del título': 18}

tercera_tabla = {'Titulo' : 'Tabla de Salarios',
                 'Leyenda': None,
                 'Tabla' : tabla_salarios,
                 'Ancho de Columnas' : [30, 30, 30],
                 'Tamaño de letra del título': 18}

leyenda_250 ="""Para calcular el salario promedio diario, se obtiene el total de
 la penúltima columna y se divide entre 1750 que es el número de días en 250 semanas,
 en las filas en las que no haya salario acumulado, significa que son fechas
 anteriores que ya no afectan el promedio"""
cuarta_tabla = {'Titulo' : 'Tabla para obtención de salario promedio diario últimas 250 semanas ($'+ str(round(SALARIO_PROMEDIO_DIARIO, 1)) + ')',
                'Leyenda': None,
                 'Tabla' : TABLA_SALARIO_PROMEDIO,
                 'Ancho de Columnas' : [30]*8,
                 'Tamaño de letra del título': 15}

pdf_bytes = test_pdfv2(segunda_tabla, tercera_tabla, cuarta_tabla)
# pdf_bytes = test_pdfv2(cuarta_tabla)

# https://py-pdf.github.io/fpdf2/Tables.html, https://rgbcolorpicker.com/
# Display the PDF in the notebook by embedding it as HTML content:

WIDTH, HEIGHT = 1200, 400
from base64 import b64encode
from IPython.display import display, HTML
base64_pdf = b64encode(pdf_bytes).decode("utf-8")
display(HTML(f'<embed height="{HEIGHT}" src="data:application/pdf;base64,{base64_pdf}" type="application/pdf" width="{WIDTH}"/>'))

In [ ]:
df_corridapagos40

,Inicio de pagos,Meses pagando,Final de pagos,Solicitud de pensión,Edad al solicitar pensión,Edad redondeada,% Pensión de acuerdo a la edad,Pago Total,Semanas totales,Salario Promedio,Pensión Alcanzada,Aumento respecto al mes anterior
0,2025-03-03,1,2025-03-31,2025-04-01,68.80,69,100.0,"$10,948.08",1965,$618.71,"$20,054.00",$0.00
1,2025-03-03,2,2025-04-30,2025-05-01,68.89,69,100.0,"$22,273.68",1969,$659.84,"$21,064.40","$1,010.40"
2,2025-03-03,3,2025-05-31,2025-06-01,68.97,69,100.0,"$33,976.80",1974,$702.59,"$22,592.80","$1,528.40"
3,2025-03-03,4,2025-06-30,2025-07-01,69.05,69,100.0,"$45,302.40",1978,$744.19,"$23,930.50","$1,337.70"
4,2025-03-03,5,2025-07-31,2025-08-01,69.14,69,100.0,"$57,005.52",1983,$786.94,"$25,681.00","$1,750.50"
5,2025-03-03,6,2025-08-31,2025-09-01,69.22,69,100.0,"$68,708.64",1987,$829.69,"$27,076.10","$1,395.10"
6,2025-03-03,7,2025-09-30,2025-10-01,69.31,69,100.0,"$80,034.24",1991,$870.82,"$28,418.30","$1,342.20"
7,2025-03-03,8,2025-10-31,2025-11-01,69.39,69,100.0,"$91,737.36",1996,$913.81,"$29,821.30","$1,403.00"
8,2025-03-03,9,2025-11-30,2025-12-01,69.47,69,100.0,"$103,062.96",2000,$954.99,"$31,165.10","$1,343.80"
9,2025-03-03,10,2025-12-31,2026-01-01,69.56,70,100.0,"$114,766.08",2004,$996.74,"$32,527.60","$1,362.50"


In [ ]:
tabla_pivot = {'Titulo' : 'Tabla Sintetizada de Pensiones Alcanzadas e Inversión total requerida empezando en Marzo de 2025',
                 'Leyenda': None,
                 'Tabla' : df_corridapagos40_simplificado,
                 'Ancho de Columnas' : [15, 25, 25, 25],
                 'Tamaño de letra del título': 18}

tabla_pivot_desglosada = {'Titulo' : 'Tabla Detallada de Pensiones Alcanzadas e Inversión total requerida',
                 'Leyenda': None,
                 'Tabla' : df_corridapagos40,
                 'Ancho de Columnas' : 12 * [10],
                 'Tamaño de letra del título': 18}

Resultados_Pivot = test_pdfv2(tabla_pivot,
                              tabla_pivot_desglosada
                              )

# Display the PDF in the notebook by embedding it as HTML content:
WIDTH, HEIGHT = 1200, 400
from base64 import b64encode
from IPython.display import display, HTML
base64_pdf = b64encode(Resultados_Pivot).decode("utf-8")
display(HTML(f'<embed height="{HEIGHT}" src="data:application/pdf;base64,{base64_pdf}" type="application/pdf" width="{WIDTH}"/>'))